In [2316]:
## PURPOSE: 

In [2317]:
# ## TODO

# (1)
# salience1, salience2, salience3 -> eu_salience DONE
# dissent1, dissent2, dissent3 -> eu_dissent DONE
# require_econ,require_pol,require_gov ->  DONE (check on email, not 100% clear that 2019 version *is* 0-10 not 1-7)
# Requires transforming scales to match

# (2) Policy variables matched DONE

# (3) Harmonise std_X X_sd DONE

# (4) Country codes/cname/country_abbrev -> Harmonise DONE!

# (5) Look at pattern of NULLS -> Maybe check against expert-level datasets?
#     # grabbed the datasets ... prob not going to be quick

# (6) Look at changes made in the 1999-2014 trend file - if they look like fixes, try to incorporate DONE
# # few issues with changed party_ids -> DONE
# # some extra columns they filled in from parlgov and the cmp_ids! -> add NEXT

# ----- pull all together


# (7) normalise everything to [0-1] and run dtype/range testing DONE

# (?) doublecheck if there's any extra inf in trend 1999-2014 trend file DONE

# types all checked, inconsistencies checked

# Handled dummy variables

# made a dictionary for the various "types" of variable

# (8) Add all notes to email to CHES

# (9) tidy previous datasets notebook, tidy this notebook, share with CHES

In [2318]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re, os, pickle

In [2319]:
dataset_folder = "C:\\Users\\Marios\\Documents\\GitHub\\Thomas Prosser project\\datasets\\"
output_folder = "C:\\Users\\Marios\\Documents\\GitHub\\Thomas Prosser project\\output\\"

In [2320]:
def could_be_int(x):
    if isinstance(x,float) or isinstance(x,int):
        return True
    else:
        try:
            int(x.strip())
            return True
        except:
            return False
        
def could_not_be_int(x):
    if isinstance(x,float) or isinstance(x,int):
        return False
    else:
        try:
            int(x.strip())
            return False
        except:
            return True        
        
def describe_messy_column(ser):

    nulls = ser.isnull().sum()
    print(nulls," nulls")
    ser = ser[ser.notnull()]
    vc = ser[ ser.apply(lambda x: could_not_be_int(x))].value_counts()
    display(vc[vc>0])
    print("min: ",ser[ ser.apply(lambda x: could_be_int(x))].min(),"max: ",ser[ ser.apply(lambda x: could_be_int(x))].max())
        

In [2321]:
# CHES_replace_dict = {
#  # CHES_2007_CAND 
#  # membership
#  'neutral':4.0,
#  # salience2
#  'little importance':2.0,
#  'some importance':3.0,
#  'great importance':4.0,
#  # require_pol
# #  'neutral':4.0,
#  # require_gov
# #  'neutral':4.0,
#  # minor
#  'strongly supports more rights for ethnic minorities':1.0,
 
    
    
#  # CHES_1999_2014
#  # eu_ep
#  'strongly opposes expanding EP powers':1.0,
#  'neutral towards expanding EP powers':4.0,
#  'strongly favors expanding EP powers':7.0,
#  # eu_fiscal
#  'strongly opposes tax harmonization':1.0,
#  'neutral towards tax harmonization':4.0,
#  'strongly favors tax harmonization':7.0,
#  # eu_intmark
#  'stongly opposes expanding EU powers on the internal market':1.0,
#  'neutral toward expanding EU powers on the internal market':4.0,
#  'stongly favors explanding EU powers on the internal market':7.0,
#  # eu_employ
#  'strongly opposes a common employment policy':1.0,
#  'neutral towards a common employment policy':4.0,
#  # eu_agri
#  'strongly opposes agricultural policy':1.0,
#  'neutral toward agricultural policy':4.0,
#  'strongly favors agricultural policy':7.0,
#  # eu_cohesion
#  "strongly opposes the EU's cohesion policy":1.0,
#  "neutral towards the EU's cohesion policy":4.0,
#  "strongly favors the EU's cohesion policy":7.0,
#  # eu_environ
#  'strongly opposes a common policy on environment':1.0,
#  'neutral towards a common policy on environment':4.0,
#  'strongly favors a common policy on environment':7.0,
#  # eu_asylum
#  'strongly opposes a common policy on political asylum':1.0,
#  'neutral towards a common policy on political asylum':4.0,
#  'strongly favors a common policy on political asylum':7.0,
#  # eu_foreign
#  'strongly opposes a common policy on political asylum':1.0,
#  'neutral towards a common policy on political asylum':4.0,
#  'strongly favors a common policy on political asylum':7.0,
#  # eu_turkey
#  'strongly opposes':1.0,
# #  'neutral':4.0,
#  # cosmo
#  # position
#  'strongly opposed':1.0,
#  'strongly in favor':7.0,
    
    
    
    
#  # CHES_1984_1999
#  # position
# 'strongly opposed to ei':1.0,
# 'neutral, no stance on ei':4.0,
# 'strongly in favor of ei':7.0,
#  # foreign
#  'strongly opposes a common foreign and security policy':1.0,
#  'neutral towards a common foreign and security policy':4.0,
#  'strongly favors a common foreign and security policy':7.0,
#  # salience1
#  'no importance, never mentioned':1.0,
#  'minor issue':2.0,
#  'important issue':3.0,
#  'one of the most important issues':4.0,
#  'the most important issue for the party':5.0,
#  # dissent2
#  'complete unity':1.0,
#  'minor dissent':2.0,
#  'significant dissent':3.0,
#  'evenly split':4.0,
#  # future
#  'unification has gone much too far':1.0,
#  'just the right amount of unification':3.0,
#  # ep
#  'strongly opposed expanding ep powers':1.0,
#  'neutral towards expanding ep powers':4.0,
#  'strongly favors expanding ep powers':7.0,
#  # fiscal
#  'strongly opposes tax harmonization':1.0,
#  'neutral towards tax harmonization':4.0,
#  'strongly favors tax harmonization':7.0,
#  # employ
#  'strongly opposes a common employment policy':1.0,
#  'neutral towards a common employment policy':4.0,
#  # cohesion
#  "strongly opposes the eu's cohesion policy":1.0,
#  "neutral towards the eu's cohesion policy":4.0,
#  "strongly favors the eu's cohesion policy":7.0,
#  # environ
#  'strongly opposes a common policy on environment':1.0,
#  'neutral towards a common policy on environment':4.0,
#  'strongly favors a common policy on environment':7.0,
#  # asylum
#  'strongly opposes a common policy on political asylum':1.0,
#  # lrgen
#  'center':5.0,
#  # lrecon
# #  'center':5.0,
#  # galtan
# #  'center':5.0,
#  'extreme tan':10.0,
#  # lr_gen
#  'extreme left':0.0,
#  'extreme right':1.0,
    
#     ## PROBLEM - neutral here is on a different scale than all other neutrals!
#  # pro_anti
# #  'anti european integration':1.0,
# #  'neutral':2.0,
# #  'pro european integration:':3.0,

# }

In [2322]:
# # amalgamate all means!

# # then look at correlation to see if any columns *really* ought to be merged

# CHES_1984_1999 = pd.read_stata(dataset_folder+"CHES"+os.sep+ "1984-1999_dataset_means.dta")
# CHES_1984_1999["source"] = "1984-1999_dataset_means.dta"
# CHES_1999_2014 = pd.read_stata(dataset_folder+"CHES"+os.sep+ "1999-2014_CHES_dataset_means-3.dta")
# CHES_1999_2014["source"] = "1999-2014_CHES_dataset_means-3.dta"
# CHES_2007_CAND = pd.read_stata(dataset_folder+"CHES"+os.sep+ "2007_CHES-candidates_dataset_means.dta")
# CHES_2007_CAND["source"] = "2007_CHES-candidates_dataset_means.dta"
# CHES_2014_UKR = pd.read_stata(dataset_folder+"CHES"+os.sep+ "2014_Candidates_Ukraine_Means.dta")
# CHES_2014_UKR["source"] = "2014_Candidates_Ukraine_Means.dta"
# CHES_2017 = pd.read_stata(dataset_folder+"CHES"+os.sep+ "CHES_means_2017.dta")
# CHES_2017["source"] = "CHES_means_2017.dta"
# CHES_2019 = pd.read_stata(dataset_folder+"CHES"+os.sep+ "CHES2019V3.dta")
# CHES_2019["source"] = "CHES2019V3.dta"

# pro_anti_replace_dict = { 'anti european integration':1.0,
#  'neutral':2.0,
#  'pro european integration':3.0,}

# # 'neutral' appears on a different scale to all other mentions
# CHES_1984_1999["pro_anti"] = CHES_1984_1999["pro_anti"].replace(pro_anti_replace_dict)

# # add year values to CHES_2019 and CHES_2014_UKR (questionnaires specify 2014/2019)
# CHES_2019["year"] = 2019
# CHES_2014_UKR["year"] = 2014

# # seems to confusion over whether party_id 1210 is PDC (party of christian democracy) or PCP (portuges communist party)
# CHES_2019.loc[103,"party_id"] = 1209


# CHES_comb = pd.concat([CHES_1984_1999,CHES_1999_2014,CHES_2007_CAND,CHES_2014_UKR,CHES_2017,CHES_2019],axis=0)
# CHES_comb = CHES_comb.replace(CHES_replace_dict)

# CHES_comb = CHES_comb.reset_index()
# CHES_comb = CHES_comb.rename(columns = {"index":"source_index"})

# # complete
# # party_id               2145
# # country                2145


# # chesversion     949
# # lrecon          949
# # eu_foreign      949
# # govt            949
# # family          949
# # electionyear    949
# # party           949
# # position        949
# # expert          949
# # year            949
# # eumember        949
# # eastwest        949
# # country         949

# # galtan          948
# # eu_salience     945
# # eu_dissent      945
# # eu_ep           942
# # eu_cohesion     941
# # vote            899
# # seat            895
# # eu_intmark      807
# # epvote          773

# # cmp_id          797

# # eu_econ_require, eu_political_require, eu_googov_require -> only 5 nonnull entries each!

# # multiculture_dissent       0
# # multiculture_salience      0
# # multiculture               0
# # eu_budget                  0

In [2323]:
# eumember_replace_dict = {"EU member state":"Yes",
#                        1:"Yes",
#                        0:"No",
#                        "other":"Other" }

# CHES_comb["eumember"] = CHES_comb["eumember"].replace(eumember_replace_dict)


# eastwest_replace_dict = { "western europe":1.0,
#                           "west":1.0,
#                           "east":0.0,
# }

# CHES_comb["eastwest"] = CHES_comb["eastwest"].replace(eastwest_replace_dict)


# govt_replace_dict = { "not in government":0.0,
#                           "in government":1.0,
# }

# CHES_comb["govt"] = CHES_comb["govt"].replace(govt_replace_dict)


# family_replace_dict = {"1. radical TAN":"rad right",
#  "2. conservative":"cons",
#  "3. liberal":"liberal",
#  "4. Christian-democratic":"christdem",
#  "5. socialist":"socialist",
#  "6. radical left":"rad left",
#  "7. green":"green",
#  "8. regionalist":"regionalist",
#  "9. no family":"no family",
#  "10. confessional":"confessional",
#  "11. agrarian/center":"agrarian",
#  "agrarian/centre":"agrarian",
#  "rad right ":"rad right",
#  "protestant":"confessional", # not 100% certain about this! theoretically, you can have confessional parties that aren't prot

# }
# CHES_comb["family"] = CHES_comb["family"].replace(family_replace_dict)


# ### all to get nicely consistent dummies most important issue variables
# df = pd.DataFrame()

# mip_replace_dict = {"public services vs taxes":"services vs taxes",
#                     "anti-elite rhetoric":"anti-elite",
#                     "ethnic minorities":"ethnic", # change these back to long-form later
#                     "EU integration":"EU",# change these back to long-form later
#                    }

# base_list = list(CHES_comb[["mip_one","mip_two","mip_three"]].replace(mip_replace_dict).stack().value_counts().index[0:18])

# base_list = pd.Series(base_list).replace(mip_replace_dict).values

# CHES_comb[["mip_one","mip_two","mip_three"]] = CHES_comb[["mip_one","mip_two","mip_three"]].replace(mip_replace_dict)



# long_form_dict ={"EU":"EU integration","ethnic":"ethnic minorities"}
    
# for cat in base_list:
#     for mip_pos in ["one","two","three"]:
#         if cat in long_form_dict.keys():
#             long_cat = long_form_dict[cat]
#         else:
#             long_cat = cat
#         df["mip_"+mip_pos+"__"+long_cat] = CHES_comb["mip_"+mip_pos]\
#                                     .apply(lambda x: mip_transform(x))\
#                                     .apply(lambda x: np.nan if x is np.nan else cat in x)\
#                                     .astype('float32')

# CHES_comb[df.columns] = df        
# CHES_comb = CHES_comb.drop(["mip_one","mip_two","mip_three"],axis=1)
# #####

# df = pd.get_dummies(CHES_comb["family"],prefix="family", prefix_sep='__')
# df.loc[CHES_comb["family"].isnull(),:] = np.nan
# CHES_comb[df.columns] = df        
# df = pd.get_dummies(CHES_comb["eumember"],prefix="eumember", prefix_sep='__')
# df.loc[CHES_comb["eumember"].isnull(),:] = np.nan
# CHES_comb[df.columns] = df        
    
# CHES_2014_UKR_country_replace_dict = {30.0:"bih",32.0:"mkd",33.0:'ser',41.0:'alb',42.0:'kos',43.0:'mne',44.0:'ukr'}
# CHES_2007_CAND_country_replace_dict = {'      30':'bih', '      31':'cro', '      32':'mkd', '      33':'ser', '      34':'tur'}    
# CHES_comb["country"] = CHES_comb["country"].replace(CHES_2014_UKR_country_replace_dict)
# CHES_comb["country"] = CHES_comb["country"].replace(CHES_2007_CAND_country_replace_dict)


# CHES_comb["party_name"] = CHES_comb["party_name"].replace("",np.nan)
# CHES_comb = CHES_comb.drop( ["cname","country_abb"] , axis=1 )

# # no entries, all from CHES_2017
# CHES_comb = CHES_comb.drop( ["multiculture_dissent","multiculture_salience","multiculture","eu_budget",] , axis=1 )

# # var00001 appears to be just an early version of party_id
# CHES_comb = CHES_comb.drop( ["var00001",] , axis=1 )


# # 1984-1999_dataset_means - dummied values for country, seems a bit premature/unnecessary to me given we have a complete column for country
# CHES_comb = CHES_comb.drop( ['aus', 'be', 'dk', 'esp', 'fin', 'fr', 'ger', 'gr', 'irl', 'it', 'nl', 'por', 'sw', 'uk',]
#                            , axis=1 )

# # confusingly, we have both a family column *and dummies* - all in this dataset
# CHES_comb = CHES_comb.drop( ['radrt', 'con', 'lib', 'cd', 'soc', 'radleft', 'green', 'reg',] , axis=1 )

# # party_name is just "party" for 2014_Candidates_Ukraine_Means
# CHES_comb.loc[CHES_comb["party_name"].notnull(),"party"] = CHES_comb.loc[CHES_comb["party_name"].notnull(),"party_name"]
# CHES_comb = CHES_comb.drop( ["party_name"] , axis=1 )

# # chesversion only present in 1999-2014_CHES_dataset_means-3.dta (1.1), changing to string to avoid any confusion
# # with numerical data
# CHES_comb["chesversion"] = CHES_comb["chesversion"].apply(lambda x: str(round(x,1)) if pd.notnull(x) else np.nan)

# # experts, eastwest, eumember (and eumember__(Yes/No/Other), 
# # party_id,cmp_id
#     # not one-to-one, mostly one-to-one, some many-to-one and
#     # *one case* where cmp has one reference for a Croatian Party of Rights and it's splinter, but CHES has two
# # country, party, partyname1, partyname2, year, family (and family__(dummied options),
# # electionyear, vote, seat, epvote,
# # source, source_index, chesversion

# # everything else is a (meaned) 'expert' evaluation

# # looking at correlations to find redundant variable
# # we get 100% correlation between lrgen and lr_gen in the 141 instances where they overlap
# # going to go ahead and amalgamate them and drop lr_gen
# CHES_comb.loc[CHES_comb["lrgen"].isnull(),"lrgen"] = CHES_comb.loc[CHES_comb["lrgen"].isnull(),"lr_gen"]*10
# CHES_comb = CHES_comb.drop("lr_gen",axis=1)

# # save it as a pickle!
# CHES_comb.to_pickle(dataset_folder+"CHES"+"CHES_comb.zip",compression='zip')

In [2324]:
# corrs = CHES_comb.corr()
# corrs = corrs.where(np.triu(np.ones(corrs.shape),k=2).astype(np.bool))
# df = corrs.stack().reset_index()
# df.columns = ['Row','Column','Value']
# # df.drop(df.index[df["Value"]==1.0],inplace=True)

# df["N"] = df.apply(lambda x: (CHES_comb[x["Row"]].notnull()&CHES_comb[x["Column"]].notnull()).sum(),axis=1)

# df.sort_values(by="Value")

,Row,Column,Value,N
4401,civlib_laworder,eu_political_require,-0.996822,5
1676,galtan,eu_political_require,-0.994686,5
6330,conflict_res,russia_role,-0.987668,12
6329,conflict_res,languages,-0.987145,12
4218,redistribution,eu_political_require,-0.984077,5
...,...,...,...,...
5385,enviro_salience,eu_googov_require,0.979922,5
2600,eu_dissent,eu_econ_require,0.985288,5
6139,corrupt_salience,eu_googov_require,0.989326,5
4332,redist_salience,eu_political_require,0.992014,5


In [2465]:
# CHES_comb[["party_id"]+[x for x in CHES_comb.columns if "blur" in x or "clear" in x]].dropna(how='all').groupby("party_id").mean().corr()

In [2326]:
# # 'eu_econ_req', 'eu_pol_req','eu_good_gov': 1-7 (2010 as 1-7, 2019 as 0-10)
# # 'require_econ', 'require_pol', 'require_gov': 0-10 (CHES_2007_CAND 1-7,CHES_2014_UKR -17)
# CHES_2019 = pd.read_stata(dataset_folder+"CHES"+os.sep+ "CHES2019V3.dta")
# CHES_2019["source"] = "CHES2019V3.dta"

# CHES_2007_CAND = pd.read_stata(dataset_folder+"CHES"+os.sep+ "2007_CHES-candidates_dataset_means.dta")
# CHES_2007_CAND["source"] = "2007_CHES-candidates_dataset_means.dta"

# CHES_2014_UKR = pd.read_stata(dataset_folder+"CHES"+os.sep+ "2014_Candidates_Ukraine_Means.dta")
# CHES_2014_UKR["source"] = "2014_Candidates_Ukraine_Means.dta"
# # 
# [x for x in CHES_2007_CAND.columns if "req" in x]
# # 2019 ['eu_econ_require', 'eu_political_require', 'eu_googov_require']
# # 2010 ['eu_econ_req', 'eu_pol_req', 'eu_good_gov']
# # 2014_UKR ['require_econ', 'require_pol', 'require_gov']
# # 2007_CAND ['require_econ', 'require_pol', 'require_gov']

# [x for x in CHES_2019.columns if "req" in x]

['eu_econ_require', 'eu_political_require', 'eu_googov_require']

In [2327]:
CHES_1999 = pd.read_stata(dataset_folder+"CHES"+os.sep+"trend files"+os.sep+"1999_CHES_dataset_means.dta")
CHES_1999["source"] = "1999_CHES_dataset_means.dta"

CHES_2002 = pd.read_stata(dataset_folder+"CHES"+os.sep+"trend files"+os.sep+"2002_CHES_dataset_means.dta")
CHES_2002["source"] = "2002_CHES_dataset_means.dta"

CHES_2006 = pd.read_stata(dataset_folder+"CHES"+os.sep+"trend files"+os.sep+"2006_CHES_dataset_means.dta")
CHES_2006["source"] = "2006_CHES_dataset_means.dta"

CHES_2010 = pd.read_stata(dataset_folder+"CHES"+os.sep+"trend files"+os.sep+"2010_CHES_dataset_means.dta")
CHES_2010["source"] = "2010_CHES_dataset_means.dta"

CHES_2014 = pd.read_stata(dataset_folder+"CHES"+os.sep+"trend files"+os.sep+"2014_CHES_dataset_means-2.dta")
CHES_2014["source"] = "2014_CHES_dataset_means-2.dta"

In [2328]:
redundant_dummies = ['gr', 'wall', 'lith', 'lat', 'sw', 'dk', 'rom', 'lib', 'por', 'est', 'be', 'nl', 'hung', 'esp',
 'reg', 'bul', 'confess', 'radrt', 'green', 'fl', 'uk', 'ger', 'irl', 'fin', 'it', 'aus', 'pol', 'bel', 'czech',
 'nofamily', 'fr', 'slovak', 'con', 'radleft', 'sv', 'sloven', 'soc', 'agrarian', 'cd', # unnecessary dummy columns
                    'pro_anti','lr_gen','var00001', # derived/replicated columns
                    'multiculture', 'multiculture_salience', 'multiculture_dissent',# 2017 flash dataset bugged empty columns
                    'eastwest', # easily calculated from country list right at the end
                    ]

column_name_replace ={"asylum":"eu_asylum",
                      "cohesion":"eu_cohesion",
                      "dissent":"eu_dissent",
                      "employ":"eu_employ",
                      "environ":"eu_environ",
                      "ep":"eu_ep",
                      "fiscal":"eu_fiscal",
                      "foreign":"eu_foreign",
                      "salience":"eu_salience",
                     
                      "agri":"eu_agri",
                      "part_id":"party_id",
                      
                      'pubserv_sal': 'pubserv_salience',
                      'dereg_sal': 'dereg_salience',
                      'redist_sal': 'redist_salience',
                      'civlib_sal': 'civlib_salience',
                      'lifestyle_sal': 'lifestyle_salience',
                      'religion_sal': 'religion_salience',
                      'immig_sal': 'immig_salience',
                      'multicult_sal': 'multicult_salience',
                      'urban_sal': 'urban_salience',
                      'cosmo_sal': 'cosmo_salience',
                      'decentral_sal': 'decentral_salience',
                      'us_sal': 'us_salience',
                      'minor_sal': 'minor_salience',                      
                      
                      "turkey":"eu_turkey",
                      "intmark":"eu_intmark",
                      
                      "position":"eu_position",
                      
                      'immasylum_salience':'multicult_salience',
                      'spendvtax':'pubserv', 'deregulation':'dereg', 'redistribution':'redist',
                      'civlib_laworder':'civlib', 'sociallifestyle':'lifestyle',
                      'religious_principle':'religion', 'immigrate_policy':'immig', 'multiculturalism':'multicult',
                      'urban_rural':'urban',  'regions':'decentral', 'ethnic_minorities':'minor',
                      'nationalism':"cosmo",
                      
                      "spendvtax_salience":"pubserv_salience",
                      "social_salience":"lifestyle_salience",
                      "relig_salience":"religion_salience",
                      "immigra_salience":"immig_salience",
                      "region_salience":"decentral_salience",
                      "ethnic_salience":"minor_salience",
                      
                      'immigrate_salience':'immig_salience','immigrate_dissent':'immig_dissent','environment':'enviro',
#                       'eu_econ_require':'eu_econ_req', 'eu_political_require':'eu_pol_req', 'eu_googov_require':'eu_good_gov',
                      'require_econ':"eu_econ_req",'require_pol':'eu_pol_req','require_gov':'eu_good_gov',
                      'religious_principles':'religion',

                      'dissep': 'eu_ep_dissent',
                      'dissintm': 'eu_intmark_dissent',
                      'dissempl': 'eu_employ_dissent',
                      'dissagri': 'eu_agri_dissent',
                      'disscohe': 'eu_cohesion_dissent',
                      'dissenv': 'eu_environ_dissent',
                      'dissasyl': 'eu_asylum_dissent',
                      'dissenlw': 'enlargw_dissent',
                      'dissenle': 'enlarge_dissent',

                      'experts':'expert',# 2017 flash dataset
                      
                      'std_position': 'position_sd',
                      'std_lrgen': 'lrgen_sd',
                      'std_lrecon': 'lrecon_sd',
                      'std_galtan': 'galtan_sd',
                      'std_salience2': 'salience2_sd',
                      'std_dissent3': 'dissent3_sd',
                      'std_membership': 'membership_sd'                      
                     }



set.intersection(set(CHES_1999), set(CHES_2002), set(CHES_2006),set(CHES_2010),set(CHES_2014))

{'country', 'galtan', 'lrecon', 'lrgen', 'source'}

In [2329]:
CHES_1999.drop(redundant_dummies,axis=1,inplace=True,errors='ignore')
CHES_1999 = CHES_1999.rename(columns = column_name_replace)
CHES_1999.shape

# common to all 1999-2014 datasets - 'country', 'galtan', 'lrecon', 'lrgen', 'source', (party_id)

# eastwest: 'west' (all!)
# eumember: 'eu member state' (all!)
# country: ['be', 'dk', 'ge', 'gr', 'esp', 'fr', 'irl', 'it', 'nl', 'uk', 'por', 'aus', 'fin', 'sv']
# year: 1999 (all!)
# source: 1999_CHES_dataset_means.dta
# expert: 5-15
# party_id: unique int id
CHES_1999["party_id"] = CHES_1999["party_id"].astype(int)
# FIXED: on the basis of changes made in the 1999_2014 trend file!
CHES_1999.loc[CHES_1999["party_id"]==710,"party_id"] = 708
# party: non-unique party abbreviation
# vote: voteshare in last national election (estimated via seat proprtion where not available for coalition partners)
    # quite a lot of 0%s
# family:['rad right ', 'cons', 'liberal', 'christdem', 'socialist', 'rad left',
#        'green', 'regionalist', 'no family', 'protestant', 'agrarian'],
# govt: categories mislabelled - read ['in government part of year', 0.5, 'in government']
    # should read ['Party not in government'(/0) ,in government part of year'(/0.5), 'in government'(/1.0)]
    # specific example party 1101 (UK Conservative) is listed as 'in government part of year'
    # entries with .5 check out (verhofstadt's Belgium 1999 6 party coalition!), as does 'in government' entries
    # note - 1999 codebook contains no 'govt' reference!
    # FIX {'in government part of year':0.0, 0.5:0.5, 'in government':1.0}
    
CHES_1999["govt"] = CHES_1999["govt"].replace( {'in government part of year':0.0, 0.5:0.5, 'in government':1.0} )

CHES_1999.shape,CHES_1999.columns

['eastwest', 'eumember', 'country', 'year', 'expert', 'party_id',
       'party', 'vote', 'family', 'govt', 'position', 'salience1', 'dissent1',
       'future', 'eu_ep', 'eu_fiscal', 'eu_employ', 'eu_cohesion',
       'eu_environ', 'eu_asylum', 'eu_foreign', 'lrgen', 'lrecon', 'galtan',
       'pro_anti', 'source'],

var1 = "galtan"
CHES_1999[var1][ CHES_1999[var1].apply(lambda x: could_not_be_int(x))].value_counts(),CHES_1999[var1][ CHES_1999[var1].apply(lambda x: could_be_int(x))].min(),CHES_1999[var1][ CHES_1999[var1].apply(lambda x: could_be_int(x))].max()

#### expert responses meaned

# position
# 1-7
# {"strongly in favor":1,
#  "neutral":4,
#  "strongly opposed":7,}

# salience1
# 1-5
# {"one of the most important issues":4,"important issue":3,"minor issue":2}

# dissent1
# 1-5
# {"complete unity":1,"minor dissent":2,'significant dissent':3}

# future
# 1-5
# {"unification has gone much too far":1, "just the right amount of unification":3}

# eu_ep
# 1-7 (8 is code for 'missing')
# {"neutral towards expanding ep powers":4,"strongly favors expanding ep powers":7, "strongly opposes expanding ep powers":1}

# eu_fiscal
# 1-7 (8 is code for 'missing')
# {"strongly opposes tax harmonization":1,"neutral towards tax harmonization":4,"strongly favors tax harmonization":7}

# eu_employ
# 1-7 (8 is code for 'missing')
# {"strongly opposes a common employment policy":1,"neutral towards a common employment policy":4}

# eu_cohesion
# 1-7 (8 is code for 'missing') - codebook has a repeated label typo here!
# {"strongly opposes the eu's cohesion policy":1,"neutral towards the eu's cohesion policy":4,"strongly favors the eu's cohesion policy":7}

# eu_environ
# 1-7 (8 is code for 'missing')
# {"strongly opposes a common policy on environment":1,"neutral towards a common policy on environment":4,"strongly favors a common policy on environment":7}

# eu_asylum
# 1-7 (8 is code for 'missing')
# {"strongly opposes a common policy on political asylum":1}

# eu_foreign
# 1-7 (8 is code for 'missing')
# {"strongly opposes a common foreign and security policy":1,"neutral towards a common foreign and security policy":4,"strongly favors a common foreign and security policy":7}

# lrgen
# 0-10
# {"center":5}

# lrecon
# 0-10
# {"center":5}

# galtan
# 0-10
# {"center":5}

# pro_anti
# drop - it's just position coded to 3 categories!

# no missing values (suspicious!) - not even party == ""


(center                3
 5.125                 0
 3.375                 0
 3.4000000953674316    0
 3.4285714626312256    0
                      ..
 6.7272725105285645    0
 6.800000190734863     0
 6.833333492279053     0
 7.0                   0
 0.6000000238418579    0
 Name: galtan, Length: 97, dtype: int64,
 0.6000000238418579,
 9.75)

In [2330]:

CHES_2002.drop(redundant_dummies,axis=1,inplace=True,errors='ignore')
CHES_2002 = CHES_2002.rename(columns = column_name_replace)
CHES_2002.shape,CHES_2002.columns
['eastwest', 'eumember', 'country', 'expert', 'year', 'party_id',
        'party', 'partyname1', 'partyname2', 'vote', 'vote1', 'family', 'govt',
        'position', 'std_position', 'salience2', 'eu_ep', 'eu_intmark',
        'eu_employ', 'eu_agri', 'eu_cohesion', 'eu_environ', 'eu_asylum',
        'eu_foreign', 'enlargw', 'enlarge', 'dissent2', 'dissep', 'dissintm',
        'dissempl', 'dissagri', 'disscohe', 'dissenv', 'dissasyl', 'dissenlw',
        'dissenle', 'lrgen', 'lrecon', 'galtan', 'std_lrgen', 'std_lrecon',
        'std_galtan', 'source'],

# NULLS: only in dissenlw        73, dissenle        98
# enlargw, enlarge (but they were coded as 99!)

# eastwest: ['west','east',]
# eumember: 0|1
# country: ['be', 'dk', 'ge', 'gr', 'esp', 'fr', 'irl', 'it', 'nl', 'uk', 'por',
#        'aus', 'fin', 'sv', 'bul', 'czech', 'hung', 'lat', 'lith', 'pol', 'rom',
#        'slovak', 'sloven']
# year: 2002 (all!)
# source: 2002_CHES_dataset_means.dta
# expert: 4-18
# party_id: unique int id
# party: non-unique party abbreviation
# partyname1, partyname2: longer string versions (original/English)
# vote: voteshare in last national election (estimated via seat proprtion where not available for coalition partners)
    # no 0%s here
# vote1: same, but for the overall coalition if party is in one (why not 'coalitional_vote'???)

# so vote1 >= vote - which is it, except in one case!!!
# CHES_2002[(CHES_2002["vote"]-CHES_2002["vote1"])>0][CHES_2002.columns[0:20]]
# 2701, https://en.wikipedia.org/wiki/2000_Romanian_general_election
# FIX! vote1 looks about right for the coalition, but obvously vote should be smaller
# Alliance was pretty asymmetric, can't see how seats were divided up, but I wouldn't imagine vote is much smaller than vote1


# family:['radical right', 'cons', 'liberal', 'christdem', 'socialist',
#        'radical left', 'green', 'region/ethnic', 'no family', 'confessional',       'agrarian']
#     FIX - categories need to be renamed!
# govt: categories mislabelled - read ['in government part of year', 0.5, 'in government']
    # SAME PROBLEM, different strings
    # should read ['Party not in government'(/0) ,in government part of year'(/0.5), 'in government'(/1.0)]
    # specific example party 1101 (UK Conservative) is listed as 'in government part of year'
    # entries with .5 check out (verhofstadt's Belgium 1999 6 party coalition!), as does 'in government' entries
    # note - 1999 codebook contains no 'govt' reference!
    # FIX {'in government part of year':0.0, 0.5:0.5, 'in government':1.0}

CHES_2002["govt"] = CHES_2002["govt"].replace( {'party in government for part of the time point of evaluation':0.0, 0.5:0.5, 'party in government during the year of evaluation':1.0} )


#### expert responses meaned

# position
# 1-7
# {"strongly in favor":1, "neutral":4, "strongly opposed":7,
#   "strongly opposed to european integration":1,"neutral towards european integration":4,"strongly in favor of european integration":7}
# - std_position




# salience2 (not salience 1 - different range)
# 1-4
# {"one of the most important issues":4,"important issue":3,"minor issue":2} salience1
# {"little importance":2,"some importance":3,"great importance":4} salience2

# dissent2 (not dissent1(1-5))
# 1-10
# {"complete unity":1,"minor dissent":2,'significant dissent':3} dissent1
# no strings in dissent2


# future DISCONTINUED
# 1-5
# {"unification has gone much too far":1, "just the right amount of unification":3}

# eu_ep
# 1-7 (8 is code for 'missing')
# {"strongly opposes expanding ep powers":1,"neutral towards expanding ep powers":4,"strongly favors expanding ep powers":7}
# {"strongly opposed expanding ep powers":1,"neutral towards expanding ep powers":4}

# eu_fiscal NOT PRESENT
# 1-7 (8 is code for 'missing')
# {"strongly opposes tax harmonization":1,"neutral towards tax harmonization":4,"strongly favors tax harmonization":7}

# eu_employ
# 1-7 (8 is code for 'missing')
# {"strongly opposes a common employment policy":1,"neutral towards a common employment policy":4}

# eu_cohesion
# 1-7 (8 is code for 'missing') - codebook has a repeated label typo here!
# {"strongly opposes the eu's cohesion policy":1,"neutral towards the eu's cohesion policy":4,"strongly favors the eu's cohesion policy":7}

# eu_environ
# 1-7 (8 is code for 'missing')
# {"strongly opposes a common policy on environment":1,"neutral towards a common policy on environment":4,"strongly favors a common policy on environment":7}

# eu_asylum
# 1-7 (8 is code for 'missing')
# expanded list
# {"strongly opposes a common policy on political asylum":1,"neutral towards a common policy on political asylum":4,"strongly favors a common policy on political asylum":7}

# eu_foreign
# 1-7 (8 is code for 'missing')
# {"strongly opposes a common foreign and security policy":1,"neutral towards a common foreign and security policy":4,"strongly favors a common foreign and security policy":7}

# lrgen
# 0-10
# {"center":5}
# - std_lrgen

# lrecon
# 0-10
# {"center":5}
# - std_lrecon

# galtan
# 0-10
# {"center":5}
# - std_galtan

# new!
# eu_intmark
# 1-7 (8 is code for missing)
# {"neutral towards internal market":4}

# eu_agri
# 1-7 (8 is code for missing)
# {"strongly opposes agricultural policy":1,"neutral toward agricultural policy":4,"strongly favors agricultural policy":7}

# enlargw (For EU-15 parties: position of the party leadership in 2002 on enlargement)
# 1-7 (8 is code for missing)
# {"neutral":4}
# FIX appears to include some "99.0" values
CHES_2002["enlargw"] = CHES_2002["enlargw"].replace(99,np.nan).replace({"neutral":4})
CHES_2002["enlargw"] = (CHES_2002["enlargw"].astype(float)-1)/6.0


# enlarge (For CEE-applicants position of the party leadership in 2002 on major domestic reforms to enter EU)
# 1-7 (8 is code for missing)
# {"neutral towards eu membership":4, "strongly supports eu membership":7}

CHES_2002["enlarge"] = CHES_2002["enlarge"].replace(99,np.nan).replace({"neutral towards eu membership":4, "strongly supports eu membership":7})
CHES_2002["enlarge"] = (CHES_2002["enlarge"].astype(float)-1)/6.0

## UNIQUE TO THIS DATASET
# I feel like these would be more readable as diss_ep, diss_intm ...
diss_vars = [x for x in CHES_2002.columns if "diss_" in x]
# diss_var_rename_dict = {k:"diss_"+k.split('diss')[1] for k in diss_vars}
# CHES_2002.rename(columns=diss_var_rename_dict,inplace=True)
## changed the names for readability/consistency
diss_vars = ['eu_ep_dissent','eu_intmark_dissent','eu_employ_dissent','eu_agri_dissent','eu_cohesion_dissent',
             'eu_environ_dissent','eu_asylum_dissent','enlargw_dissent','enlarge_dissent']
     
# diss variables: "has this issue caused divisions within the leadership in 2002?"
# *number of experts checking box* (no constraint on how many boxes they can check)

CHES_2002[diss_vars] = CHES_2002[diss_vars].apply(lambda x: x/CHES_2002["expert"])
# normalised
        

# var1 = "diss_ep"
# CHES_2002[var1][ CHES_2002[var1].apply(lambda x: could_not_be_int(x))].value_counts(),CHES_2002[var1][ CHES_2002[var1].apply(lambda x: could_be_int(x))].min(),CHES_2002[var1][ CHES_2002[var1].apply(lambda x: could_be_int(x))].max()


In [2331]:

CHES_2006.drop(redundant_dummies,axis=1,inplace=True,errors='ignore')
CHES_2006 = CHES_2006.rename(columns = column_name_replace)
CHES_2006.shape,CHES_2006.columns

['eastwest', 'eumember', 'country', 'country_abb', 'year', 'expert',
        'party_id', 'party', 'partyname1', 'partyname2', 'vote', 'vote1',
        'family', 'intgroup', 'epgroup', 'govt', 'govt1', 'govt2', 'govt3',
        'position', 'std_position', 'salience2', 'std_salience2', 'dissent3',
        'std_dissent3', 'benefit', 'eu_ep', 'eu_intmark', 'eu_cohesion',
        'eu_foreign', 'eu_turkey', 'lrgen', 'std_lrgen', 'lrecon', 'std_lrecon',
        'galtan', 'std_galtan', 'var00001', 'pubserv', 'pubserv_salience', 'dereg',
        'dereg_salience', 'redist', 'redist_salience', 'civlib', 'civlib_salience',
        'lifestyle', 'lifestyle_salience', 'religion', 'religion_salience', 'immig',
        'immig_salience', 'multicult', 'multicult_salience', 'urban', 'urban_salience',
        'cosmo', 'cosmo_salience', 'decentral', 'decentral_salience', 'us', 'us_salience',
        'minor', 'minor_salience', 'source'],



# eastwest: ['west','east',]
# eumember: ['eu member state','other'] (other= bulgaria and romania)
# country: ['be', 'dk', 'ge', 'gr', 'esp', 'fr', 'irl', 'it', 'nl', 'uk', 'por',
#        'aus', 'fin', 'sv', 'bul', 'cz', 'est', 'hun', 'lat', 'lith', 'pol',
#        'rom', 'slo', 'sle'],
# country_abb: More or less the same as country but in capitals??
# year: 2006 (all!)
# source: 2006_CHES_dataset_means.dta    
# expert: 4-14
# party_id: unique int id
# party: non-unique party abbreviation
# partyname1, partyname2: longer string versions (original/English)
# vote: voteshare in last national election (estimated via seat proprtion where not available for coalition partners)
    # no 0%s here
# vote1: same, but for the overall coalition if party is in one (why not 'coalitional_vote'???)
# family:['radical right', 'cons', 'liberal', 'christdem', 'socialist',
#        'radical left', 'green', 'region/ethnic', 'no family', 'confessional',       'agrarian']
#     FIX - categories need to be renamed!
# govt: categories mislabelled - read ['in government part of year', 0.5, 'in government']
    # SAME PROBLEM, different strings
    # should read ['Party not in government'(/0) ,in government part of year'(/0.5), 'in government'(/1.0)]
    # specific example party 1101 (UK Conservative) is listed as 'in government part of year'
    # entries with .5 check out (verhofstadt's Belgium 1999 6 party coalition!), as does 'in government' entries
    # note - 1999 codebook contains no 'govt' reference!
    # FIX {'in government part of year':0.0, 0.5:0.5, 'in government':1.0}
    
CHES_2006["govt"] = CHES_2006["govt"].replace( {'in government part of year':0.0, 0.5:0.5, 'in government':1.0} )
# govt1: months in government in 2006
CHES_2006["govt1"] = CHES_2006["govt1"]/12
# govt2: no notes in codebook - could it be days? Ranges from 0 -288 - floats - 45 zeroes not 108 (parties never in gov)??
# govt3: similarly confusing!

# intgroup: membership in international political group (2007)
# ['socialist international', 'centrist democratic international',
#        'liberal international', 'global greens',
#        'international democrat union', 'alliance of democrats',
#        'cdi (2) and idu (5)', 'no affiliation']

# epgroup: 
# [        'pes',      'epp-ed',        'alde',  'greens-efa',
#                'uen',     'gue-ngl',          'id',           8.0,       'non-incrits']
# {'pes':"Party of European Socialists (PES)",'epp-ed':"European People’s Party – European Democrats (EPP-ED)",
# 'alde':"Alliance for Liberals and Democrats for Europe (ALDE)",'greens-efa':"European Greens – European Free Alliance (Greens-EFA)",
# 'uen':"Union for Europe of the Nations (UEN)",'gue-ngl':"European United Left – Nordic Green Left (GUE-NGL)",
# 'id':"Independence/Democracy (ID)",'non-incrits':"Non-Incrits (NI)",8.0:"no group"}
            # no group parties, almost all(?) far right
    
    
#### expert responses meaned

# position
# 1-7
# {"strongly in favor":1, "neutral":4, "strongly opposed":7,
#   "strongly opposed to european integration":1,"neutral towards european integration":4,"strongly in favor of european integration":7}
# {"strongly opposed":1,"neutral":4,"strongly in favor":7}
# - std_position

# salience2 (not salience 1 - different range)
# 1-4
# {"one of the most important issues":4,"important issue":3,"minor issue":2} salience1
# {"little importance":2,"some importance":3,"great importance":4} salience2
# -std_salience2



# dissent3 (not dissent1(1-5) or dissent2(1-10))
# 0-10
# {"complete unity":1,"minor dissent":2,'significant dissent':3} dissent1
# no strings in dissent2
# {"party was completely united":0}
# - std_dissent3

# future NOT PRESENT
# 1-5
# {"unification has gone much too far":1, "just the right amount of unification":3}



# benefit
# party leadership’s stance on whether the country has benefited from EU membership (Q4)
# [Note this question corresponds to the standard Eurobarometer “benefit” question.]
# ['benefited','neither benefited nor lost','not benefited']
{'benefited':1,'neither benefited nor lost':2,'not benefited':3, 0.5: 2}
# partyid: 828 - .5 IdV	Di Pietro-Italia dei Valori	Italy of Values - has a value of 0.5, shouldn't be possible
# I'm guessing this should be the intermediate value 2??? BUG!

# eu_ep
# 1-7 (8 is code for 'missing')
# {"strongly opposes expanding ep powers":1,"neutral towards expanding ep powers":4,"strongly favors expanding ep powers":7}
# {"strongly opposed expanding ep powers":1,"neutral towards expanding ep powers":4}
# 7 NULLS???

# eu_fiscal NOT PRESENT
# 1-7 (8 is code for 'missing')
# {"strongly opposes tax harmonization":1,"neutral towards tax harmonization":4,"strongly favors tax harmonization":7}

# eu_employ NOT PRESENT
# 1-7 (8 is code for 'missing')
# {"strongly opposes a common employment policy":1,"neutral towards a common employment policy":4}

# eu_cohesion
# 1-7 (8 is code for 'missing') - codebook has a repeated label typo here!
# {"strongly opposes the eu's cohesion policy":1,"neutral towards the eu's cohesion policy":4,"strongly favors the eu's cohesion policy":7}
# {'neutral':4,strongly favors ':7,}
# 8 NULLS???

# eu_environ NOT PRESENT
# 1-7 (8 is code for 'missing')
# {"strongly opposes a common policy on environment":1,"neutral towards a common policy on environment":4,"strongly favors a common policy on environment":7}

# eu_asylum NOT PRESENT
# 1-7 (8 is code for 'missing')
# expanded list
# {"strongly opposes a common policy on political asylum":1,"neutral towards a common policy on political asylum":4,"strongly favors a common policy on political asylum":7}

# eu_foreign
# 1-7 (8 is code for 'missing')
# {"strongly opposes a common foreign and security policy":1,"neutral towards a common foreign and security policy":4,"strongly favors a common foreign and security policy":7}
# {"strongly opposes":1,neutral":4,}

# lrgen
# 0-10
# {"center":5,"extreme right":10}
# - std_lrgen

# lrecon
# 0-10
# {"center":5}
# - std_lrecon

# galtan
# 0-10
# {"center":5}
# - std_galtan
# 1 NULL???

# eu_intmark
# 1-7 (8 is code for missing)
# {"neutral towards internal market":4}
# {"strongly opposes":1,neutral":4}

# eu_agri NOT PRESENT
# 1-7 (8 is code for missing)
# {"strongly opposes agricultural policy":1,"neutral toward agricultural policy":4,"strongly favors agricultural policy":7}
    
# eu_turkey 8 nulls 1-7
# {"strongly opposes":1,"neutral":4}
        
# pubserv 0-10
# pubserv_salience 0-10
# 2 NULLS???

# dereg
# 3 NULLS
# dereg_sal
# 2 NULLS 

# redist 2 NULLS
{"strongly favors redistribution":0}
# redist_salience 4 NULLS

# civlib 2 NULLS
# civlib_salience 2 NULLS

# lifestyle 3 NULLS
{"strongly opposes liberal policies":10}
# lifestyle_salience 3 NULLS

# religion 3 NULLS
{"strongly supports religious principles in politics":10}
# religion_salience 3 NULLS

# immig 7 NULLS
# immig_salience 4 NULLS

# multicult 10 NULLS
{"strongly favors assimilation":0}
# multicult_salience 5 NULLS

# urban 7 NULLS
# urban_salience 7 NULLS

# cosmo 6 NULLS
{"strongly advocates nationalism":0}
# cosmo_say 6 NULLS

# decentral 4 NULLS
# decentral_salience 10 NULLS

# us 2 NULLS
{"strongly opposes strong us leadership in world affairs":0}
# us_salience 2 NULLS

# minor 3 NULLS 0-10
{"strongly supports more rights for ethnic minorities":0,"strongly opposes more rights for ethnic minorities":10}
# minor_salience 3 NULLS

# all _salience variables 0-10
{"extremely important":10}

### BUG! standard deviation of 5 ... on a variable that's supposed to run from 1-4 is not possible
CHES_2006.loc[ CHES_2006["salience2_sd"]==5.0, "salience2_sd"] = np.nan


describe_messy_column(CHES_2006["dissent3"])

#### LOOOK AT PATTERN OF MISSING VALUES!

0  nulls


party was completely united    1
Name: dissent3, dtype: int64

min:  0.2199999988079071 max:  8.670000076293945


In [2332]:
CHES_2010.drop(redundant_dummies,axis=1,inplace=True,errors='ignore')
CHES_2010 = CHES_2010.rename(columns = column_name_replace)
CHES_2010.shape,CHES_2010.columns

['country', 'cname', 'party_id', 'party_name', 'position', 'eu_salience',
       'eu_dissent', 'eu_benefit', 'eu_ep', 'eu_intmark', 'eu_cohesion',
       'eu_foreign', 'eu_turkey', 'lrgen', 'lrecon', 'galtan', 'pubserv',
       'pubserv_salience', 'dereg', 'dereg_salience', 'redist',
       'redist_salience', 'civlib', 'civlib_salience', 'lifestyle',
       'lifestyle_salience', 'religion', 'religion_salience', 'immig',
       'immig_salience', 'multicult', 'multicult_salience', 'urban',
       'urban_salience', 'environment', 'enviro_salience', 'decentral',
       'decentral_salience', 'international_security',
       'international_salience', 'minor', 'minor_salience', 'eu_econ_req',
       'eu_pol_req', 'eu_good_gov', 'source']

# country: suddenly is now code numbers!
# cname: now the lower case short country string previously in country
# party_id: unique int id
CHES_2010["party_id"] = CHES_2010["party_id"].astype(int)
# FIXED: on the basis of changes made in the 1999_2014 trend file!
CHES_2010.loc[CHES_2010["party_id"]==615,"party_id"] = 621
CHES_2010.loc[CHES_2010["party_id"]==841,"party_id"] = 836
CHES_2010.loc[CHES_2010["party_id"]==1019,"party_id"] = 1006
# party_name: short abbreviated form non-unique
# year not present
CHES_2010["year"] = 2010

# position 1-7
# eu_salience 1-4 (same as salience2)
# eu_dissent supposed to be same as dissent3 0-10 ... but we don't see any values above 7 (same questionnaire goes up to 10)
# eu_benefit 1-3 17NULLS
# eu_ep 1-7 17NULLS
# eu_intmark 1-7 17NULLS
# eu_cohesion 1-7 20NULLS
# eu_foreign 1-7 20NULLS
# eu_turkey 1-7 18NULLS

# lrgen 0-10
# lrecon 0-10
# galtan 0-10

# pubserv|dereg|redist|civlib|religion|immig|multicult|urban|environment|decentral|international_security|minor 0-10
# ditto these + _salience 0-10
# only 1NULL in minor_salience

# variable naming inconsistency -> _salience or _sal?
# BUG immasylum_salience instead of multiculturalism_salience!!!

# EU_ECON_REQ, EU_POL_REQ, EU_GOOD_GOV 1-7
# only asked of Croatia and Turkey (17 non-nulls each), and only in 2010


# CHES_2010["position"].value_counts()
describe_messy_column(CHES_2010["eu_good_gov"])
# CHES_2010["eu_benefit"].value_counts()



222  nulls


Series([], Name: eu_good_gov, dtype: int64)

min:  3.0 max:  7.0


In [2333]:
CHES_2014.drop(redundant_dummies,axis=1,inplace=True,errors='ignore')
CHES_2014 = CHES_2014.rename(columns = column_name_replace)
CHES_2014.shape,CHES_2014.columns

['country', 'cname', 'party_id', 'party_name', 'eu_position',
        'eu_position_sd', 'eu_salience', 'eu_dissent', 'eu_benefit', 'eu_ep',
        'eu_intmark', 'eu_cohesion', 'eu_foreign', 'eu_turkey', 'eu_budgets',
        'lrgen', 'lrgen_sd', 'lrecon', 'lrecon_salience', 'lrecon_sd', 'galtan',
        'galtan_salience', 'galtan_sd', 'pubserv', 'dereg', 'redist',
        'econ_interven', 'civlib', 'lifestyle', 'religion', 'immig',
        'multicult', 'urban', 'environment', 'decentral',
        'international_security', 'minor', 'cosmo', 'antielite_salience',
        'corrupt_salience', 'mip_one', 'mip_two', 'mip_three', 'source']

# country: suddenly is now code numbers!
# cname: now the lower case short country string previously in country
# party_id: unique int id
CHES_2014["party_id"] = CHES_2014["party_id"].astype(int)


# party_name: short abbreviated form non-unique
# year not present
CHES_2014["year"] = 2014
# eu_position: 1-7 (previous 'position' column names updated for consistency)
# eu_position_sd: 22 NULLS
# eu_salience,eu_dissent: 0-10
# eu_benefit: 1-3
# eu_ep: 1-7 1 NULL
# eu_intmark: 1-7
# eu_cohesion: 1-7 1 NULL
# eu_foreign: 1-7 1 NULL
# eu_turkey: 1-7 2 NULLS
# eu_budgets: 1-7 1 NULL
# lrgen: 0-10
# lrgen_sd: 22 NULLs
# lrecon: 0-10
# lrecon_sd: 23 NULLS
# lrecon_salience: 0-10
# galtan: 0-10
# galtan_sd: 22 NULLS
# galtan_salience: 0-10

# policy position - all 0-10
# 'pubserv', 'dereg', 'redist', 'econ_interven', 'civlib', 'lifestyle', 'religion', 'immig',
# 'multicult', 'urban', 'environment', 'decentral', 'international_security', 'minor', 'cosmo',
# NULLS
# immig                      1
# environment                1
# decentral                  2
# international_security     1

# antielite_salience: 0-10
# corrupt_salience: 0-10

# most important problem structures already solved above
#  'mip_one', 'mip_two', 'mip_three'
# NULLS
# mip_one                   22
# mip_two                   24
# mip_three                 26


CHES_2014["party_id"].value_counts()
describe_messy_column(CHES_2014["mip_one"])

22  nulls


public services vs taxes                                                              66
redistribution                                                                        31
nationalism                                                                           20
environment                                                                           17
anti-elite rhetoric                                                                   16
corruption                                                                            13
immigration                                                                           12
EU integration                                                                        12
ethnic minorities                                                                      9
decentralization                                                                       9
religious principles                                                                   6
deregulation         

min:  nan max:  nan


In [2334]:
CHES_1984_1999 = pd.read_stata(dataset_folder+"CHES"+os.sep+ "1984-1999_dataset_means.dta")
CHES_1984_1999["source"] = "1984-1999_dataset_means.dta"

# part of this data comes from here:
# https://www.lsu.edu/faculty/lray2/data/data.html

CHES_1984_1999["lrgen"] = CHES_1984_1999["lrgen"].replace({"center":5}).astype(float)
CHES_1984_1999["lr_gen"] = CHES_1984_1999["lr_gen"].replace({"extreme left":0.0,"extreme right":1.0}).astype(float)
CHES_1984_1999.loc[CHES_1984_1999["lrgen"].isnull(),"lrgen"] = CHES_1984_1999.loc[CHES_1984_1999["lrgen"].isnull(),"lr_gen"]*10

CHES_1984_1999.drop(redundant_dummies,axis=1,inplace=True,errors='ignore')
CHES_1984_1999 = CHES_1984_1999.rename(columns = column_name_replace)

# dissent2:  1-5 !!!CODEBOOK says Dissent1 and this is obviously the dissent 1 version
CHES_1984_1999 = CHES_1984_1999.rename(columns = {"dissent2":"dissent1"})

CHES_1984_1999.shape,CHES_1984_1999.columns

# CHES_1984_1999.isnull().sum()[CHES_1984_1999.isnull().sum()>0]
# NULLS
# expert         545
# var00001       545
# future         545
# eu_ep          545
# eu_fiscal      545
# eu_employ      545
# eu_cohesion    545
# eu_environ     545
# eu_asylum      545
# eu_foreign     545
# lrgen          545
# lrecon         545
# galtan         545

# the above were all only sampled in year 1999

# lr_gen         132
# no obvious pattern

# pro_anti         2
# position         2
# salience1        2
# dissent2         5


# 'eastwest', 'eumember', 'country', 'expert', 'party_id', 'year',
#         'party', 'vote', 'family', 'position', 'salience1', 'dissent2',
#         'future', 'eu_ep', 'eu_fiscal', 'eu_employ', 'eu_cohesion',
#         'eu_environ', 'eu_asylum', 'eu_foreign', 'lrgen', 'lrecon', 'galtan',
 
# eastwest: "western europe" (all!)
# eumember: 0|1 (mostly 1, presumably == EU member)
# country: ['be', 'dk', 'ge', 'gr', 'esp', 'fr', 'irl', 'it', 'nl', 'uk', 'por', 'aus', 'fin', 'sv']
# expert:  5-15
# party_id: not-unique (covers multiple years)
# year: 1984|1988|1992|1996|1999
# party: abbreviated party initials not unique
# vote: voteshare - contains 101 0.0's ... which doesn't sound right (e.g. maybe code for 'missing value')
# family:['rad right ', 'cons', 'liberal', 'christdem', 'socialist', 'rad left',
#        'green', 'regionalist', 'no family', 'protestant', 'agrarian']
# position: 1-7
# {"strongly opposed to ei":1,"neutral, no stance on ei":4,"strongly in favor of ei":7}
# {"strongly in favor":1, "neutral":4, "strongly opposed":7,
#   "strongly opposed to european integration":1,"neutral towards european integration":4,"strongly in favor of european integration":7}
# {"strongly opposed":1,"neutral":4,"strongly in favor":7}
# - std_position


# salience1: 1-5
{"no importance, never mentioned":1,
 "minor issue":2,
 "important issue":3,
 "one of the most important issues":4,
 "the most important issue for the party":5,
}
# dissent2:  1-5 !!!CODEBOOK says Dissent1 and this is obviously the dissent 1 version
{"complete unity":1,
"minor dissent":2,
"significant dissent":3,
"evenly split":4,}
# future: 1-5
{"unification has gone much too far":1,
"just the right amount of unification":3}

# eu_ep: 1-7
{"strongly opposed expanding ep powers":1,
"neutral towards expanding ep powers":4,
"strongly favors expanding ep powers":7}

# eu_fiscal: 1-7
{"strongly opposes tax harmonization":1,
"neutral towards tax harmonization":4,
"strongly favors tax harmonization":7}

# eu_employ: 1-7
{"strongly opposes a common employment policy":1,
 "neutral towards a common employment policy":4,}

# eu_cohesion: 1-7
{"strongly opposes tax harmonization":1,
 "neutral towards tax harmonization":4,
 "strongly favors tax harmonization":7}

# eu_environ: 1-7
{"strongly opposes a common policy on environment":1,
"neutral towards a common policy on environment":4,
"strongly favors a common policy on environment":7}

# eu_asylum: 1-7
{"strongly opposes a common policy on political asylum":1}

# eu_foreign: 1-7
{"strongly opposes a common foreign and security policy":1,
 "neutral towards a common foreign and security policy":4,
 "strongly favors a common foreign and security policy":7}
# 'lrgen', 'lrecon', 'galtan',
# lrgen: 0-10
# lrecon: 0-10
# galtan: 0-10

describe_messy_column(CHES_1984_1999["galtan"])
# CHES_1984_1999["family"].value_counts()
# CHES_1984_1999["family"].cat.categories

545  nulls


center    3
Name: galtan, dtype: int64

min:  0.6000000238418579 max:  9.75


In [2335]:
CHES_2019 = pd.read_stata(dataset_folder+"CHES"+os.sep+ "CHES2019V3.dta")
CHES_2019["source"] = "CHES2019V3.dta"

CHES_2019.drop(redundant_dummies,axis=1,inplace=True,errors='ignore')
CHES_2019 = CHES_2019.rename(columns = column_name_replace)
CHES_2019.shape,CHES_2019.columns

['country', 'eastwest', 'party', 'party_id', 'eu_position',
        'eu_position_sd', 'eu_salience', 'eu_dissent', 'eu_blur', 'eu_cohesion',
        'eu_foreign', 'eu_intmark', 'eu_budgets', 'eu_asylum', 'lrgen',
        'lrecon', 'lrecon_sd', 'lrecon_salience', 'lrecon_dissent',
        'lrecon_blur', 'galtan', 'galtan_sd', 'galtan_salience',
        'galtan_dissent', 'galtan_blur', 'immig', 'immig_salience',
        'immig_dissent', 'multicult', 'multicult_salience', 'multicult_dissent',
        'redist', 'redist_salience', 'enviro', 'enviro_salience', 'pubserv',
        'dereg', 'econ_interven', 'civlib', 'lifestyle', 'religion', 'minor',
        'cosmo', 'urban', 'protectionism', 'decentral', 'russian_interference',
        'anti_islam_rhetoric', 'people_vs_elite', 'antielite_salience',
        'corrupt_salience', 'members_vs_leadership', 'eu_econ_req',
        'eu_pol_req', 'eu_good_gov', 'source'],

# country: ['be', 'dk', 'ge', 'gr', 'esp', 'fr', 'irl', 'it', 'nl', 'uk', 'por',
#        'aus', 'fin', 'sv', 'bul', 'cz', 'est', 'hun', 'lat', 'lith', 'pol',
#        'rom', 'slo', 'sle', 'cro', 'tur', 'nor', 'swi', 'mal', 'lux', 'cyp', 'ice']
# eastwest: 0.0|1.0
# party: party initials not unique
# party_id: int unique id
CHES_2019["party_id"] = CHES_2019["party_id"].astype(int)
# year not present
CHES_2019["year"] = 2019
# eu_position: 1-7
# - eu_position_sd
# eu_salience: 0-10
# eu_dissent: 0-10 2 NULLS
# eu_blur: 0-10 1 NULL

# eu_cohesion: 1-7 30 NULLS
# eu_foreign: 1-7 31 NULLS
# eu_intmark: 1-7 30 NULLS
# eu_budgets: 1-7 31 NULLS
# eu_asylum: 1-7 31 NULLS

# lrgen: 0-10
# lrecon: 0-10
# - lrecon_sd
# - lrecon_salience: 0-10
# - lrecon_dissent: 0-10 2 NULLs
# galtan: 0-10
# -galtan_sd
# -galtan_salience: 0-10
# -galtan_dissent: 0-10 8 NULLS
# -galtan_blur: 0-10 2 NULLS

# policy positions
# ['immig', 'immig_salience',
#         'immig_dissent', 'multicult', 'multicult_salience', 'multicult_dissent',
#         'redist', 'redist_salience', 'enviro', 'enviro_salience', 'pubserv',
#         'dereg', 'econ_interven', 'civlib', 'lifestyle', 'religion', 'minor',
#         'cosmo', 'urban', 'protectionism', 'decentral', 'russian_interference',
#         'anti_islam_rhetoric'] - all within 0-10, no strings
# NULLS
# immig_dissent          1
# multicult_salience     1
# multicult_dissent      1
# redist                 1
# pubserv                1
# civlib                 1
# minor                  1
# cosmo                  1
# protectionism          1
# decentral              4
# anti_islam_rhetoric    6

# people_vs_elite: 0-10
# antielite_salience: 0-10
# corrupt_salience: 0-10
# members_vs_leadership: 0-10 1NULLS

# eu_econ_req: 0-10 272NULLS
# eu_pol_req: 0-10 272NULLS
# eu_good_gov: 0-10 272NULLS

# describe_messy_column(CHES_2019["eu_good_gov"])
# CHES_2019["party_id"].value_counts()
# CHES_2019["country"].cat.categories

# CHES_2019["country"].value_counts().index

# 2019 eu member states 
other_eu_2019 = ['uk','tur',] ## if you think of 'other' as 'in process of exiting/entering', makes sense to include uk
not_eu_2019 = ['isl','swi','nor',]
eu_2019 = ['cro', 'esp', 'nl', 'slo', 'be', 'lith', 'dk', 'bul', 'ge',
                 'fr', 'irl', 'lat', 'cz', 'pol', 'nor', 'sle', 'por', 'gr',
                  'swi', 'it',  'fin', 'sv', 'cyp', 'ice', 'hun', 'rom',
                  'lux', 'est', 'aus',  'mal',] 

eumember_dict = {k:"Yes" for k in CHES_2019["country"].unique()}
eumember_dict["uk"] = "Other"
eumember_dict["tur"] = "Other"
eumember_dict["isl"] = "No"
eumember_dict["swi"] = "No"
eumember_dict["nor"] = "No"

CHES_2019["eumember"] = CHES_2019["country"].replace(eumember_dict)

In [2336]:
CHES_2007_CAND = pd.read_stata(dataset_folder+"CHES"+os.sep+ "2007_CHES-candidates_dataset_means.dta")
CHES_2007_CAND["source"] = "2007_CHES-candidates_dataset_means.dta"

CHES_2007_CAND.drop(redundant_dummies,axis=1,inplace=True,errors='ignore')
CHES_2007_CAND = CHES_2007_CAND.rename(columns = column_name_replace)
CHES_2007_CAND.shape,CHES_2007_CAND.columns
# 39 x 30

# NULLS - none, which is ... suspect

# ['eumember', 'country', 'country_abb', 'year', 'expert', 'party_id',
#         'party', 'partyname1', 'partyname2', 'membership', 'std_membership',
#         'salience2', 'std_salience2', 'dissent3', 'std_dissent3',
#         'require_econ', 'require_pol', 'require_gov', 'lrgen', 'std_lrgen',
#         'lrecon', 'std_lrecon', 'galtan', 'std_galtan', 'pubserv', 'redist',
#         'religion', 'cosmo', 'minor', 'source']

# eumember: 'other' (all!)
CHES_2007_CAND["eumember"] = "Other"
# country: codes [30,31,32,33,34]
# country_abb: [CRO,SER,MKD,BID,TUR] - like cname, but allcaps
# year: 2007 (all!)
# expert: 5-9
# party_id: unique int id
# party: party initials not unique
# partyname1: original language full name unique(in this small dataset)
# partyname2: english version - not unique


# membership: 1-7
{"neutral":4}
# - std_membership
# salience2: 1-4
{"little importance":2,
 "some importance":3,
 "great importance":4,}
# - std_salience2
# dissent3: 0-10
# - std_dissent3

# These match the eu_req_econ/pol/good_gov variables but the scale needs to be transformed from 1-7 -> 0-10
# require_econ: 1-7
# require_pol: 1-7
{"neutral":4}
# require_gov: 1-7
{"neutral":4}

# lrgen: 0-10
{"center":5}
# -std_lrgen
# lrecon: 0-10
{"center":5}
# -std_lrecon
# galtan: 0-10
# -std_galtan

# policies
# pubserv: 0-10
# redist: 0-10
# religion: 0-10
# cosmo: 0-10
# minor: 0-10
{"strongly supports more rights for ethnic minorities":0}




describe_messy_column(CHES_2007_CAND["pubserv"])
# CHES_2007_CAND["membership"].value_counts()
# CHES_2007_CAND["country"].cat.categories

CHES_2007_CAND.shape,CHES_2007_CAND.columns

0  nulls


Series([], Name: pubserv, dtype: int64)

min:  1.0 max:  8.25


((39, 30),
 Index(['eumember', 'country', 'country_abb', 'year', 'expert', 'party_id',
        'party', 'partyname1', 'partyname2', 'membership', 'membership_sd',
        'salience2', 'salience2_sd', 'dissent3', 'dissent3_sd', 'eu_econ_req',
        'eu_pol_req', 'eu_good_gov', 'lrgen', 'lrgen_sd', 'lrecon', 'lrecon_sd',
        'galtan', 'galtan_sd', 'pubserv', 'redist', 'religion', 'cosmo',
        'minor', 'source'],
       dtype='object'))

In [2337]:
CHES_2014_UKR = pd.read_stata(dataset_folder+"CHES"+os.sep+ "2014_Candidates_Ukraine_Means.dta")
CHES_2014_UKR["source"] = "2014_Candidates_Ukraine_Means.dta"

CHES_2014_UKR.drop(redundant_dummies,axis=1,inplace=True,errors='ignore')
CHES_2014_UKR = CHES_2014_UKR.rename(columns = column_name_replace)
CHES_2014_UKR.shape,CHES_2014_UKR.columns
# 62 x 38
# NULLS
# Ukraine specific questions
# conflict_res     50
# region_status    50
# languages        50
# russia_role      50
# oligarchs        50





# 'country', 'cname', 'party_id', 'party_name', 'membership',
#        'eu_salience', 'eu_dissent', 'require_econ', 'require_pol',
#        'require_gov', 'lrgen', 'lrecon', 'lrecon_salience', 'galtan',
#        'galtan_salience', 'pubserv', 'dereg', 'redist', 'econ_interven',
#        'civlib', 'lifestyle', 'religion', 'immig', 'multicult', 'urban',
#        'enviro', 'decentral', 'international_security', 'minor', 'cosmo',
#        'antielite_salience', 'corrupt_salience', 'conflict_res',
#        'region_status', 'languages', 'russia_role', 'oligarchs', 'source']

# country: codes[30,32,33,41,42,43,44]
# cname: [bih,ukr,ser,mne,kos,alb,mkd]
# no year column - but implicitly 2014!
CHES_2014_UKR["year"] = 2014
# no expert column
# party_id: unique int id
CHES_2014_UKR["party_id"] = CHES_2014_UKR["party_id"].astype(int)
# party_name: party initials not unique
# eumember: not present
CHES_2014_UKR["eumember"] = "Other"


# membership: 1-7
# eu_salience: 0-10
# CODEBOOK BUG erroneously says there should be an eu_salience_sd
# eu_dissent: 0-10
# require_econ,require_pol,require_gov: 1-7

# lrgen,lrecon,galtan:0-10
# lrecon_salience,galtan_salience: 0-10
# CODEBOOK BUG incorrectly says there should be lrgen_sd,lrecon_sd,galtan_sd

# policies - all fine 0-10
# 'pubserv', 'dereg', 'redist', 'econ_interven',
#        'civlib', 'lifestyle', 'religion', 'immig', 'multicult', 'urban',
#        'enviro', 'decentral', 'international_security', 'minor', 'cosmo',

# salience - both fine 0-10
#        'antielite_salience', 'corrupt_salience',

# Ukraine specific: all fine 0-10
# 'conflict_res', 'region_status', 'languages', 'russia_role', 'oligarchs'

describe_messy_column(CHES_2014_UKR["oligarchs"])
# CHES_2014_UKR["membership"].value_counts()
# CHES_2014_UKR["country"].cat.categories


# CHES_2014_UKR.isnull().sum()[CHES_2014_UKR.isnull().sum()>0]
# CHES_2014_UKR.columns


50  nulls


Series([], Name: oligarchs, dtype: int64)

min:  4.0625 max:  9.333333


In [2338]:
CHES_2017 = pd.read_stata(dataset_folder+"CHES"+os.sep+ "CHES_means_2017.dta")
CHES_2017["source"] = "CHES_means_2017.dta"

# electionyear[new],seat[new],
# bugged columns 'multiculture', 'multiculture_salience', 'multiculture_dissent' - empty
# ditto ['eu_position','eu_budget','immigrate_salience']
# party_name column also bugged - only ""
CHES_2017 = CHES_2017.drop(["party_name",'eu_position','eu_budget','immigrate_salience'],axis=1)

# experts -> expert
# _clear -> _blur? - actually very good case for this 60-70% negative correlation between X_clear and X_blur
# it is only 6 variables: lrecon_clear	galtan_clear	galtan_new_clear	eu_blur	lrecon_blur	galtan_blur
# only 2 overlapping pairs

CHES_2017.drop(redundant_dummies,axis=1,inplace=True,errors='ignore')
CHES_2017 = CHES_2017.rename(columns = column_name_replace)

CHES_2017.shape,CHES_2017.columns

# NULLS
# vote                  6
# seat                  5
# eu_asylum             1
# eu_budgets            1
# galtan_clear          1
# immig                 1
# immig_dissent         1
# multicult             1
# multicult_salience    1
# multicult_dissent     1
# galtan_new_clear      1

# country: ['ger', 'gr', 'esp', 'fr', 'it', 'nl', 'uk', 'por', 'swe', 'cz', 'est', 'hun', 'pol', 'slo']
# year: 2017 (all!)
# expert: 12-22
CHES_2017["expert"] = CHES_2017["expert"].astype(int)
# party_id: unique int id
CHES_2017["party_id"] = CHES_2017["party_id"].astype(int)
# party: initials not unique
# vote: voteshare (0.01-42.4) voteshare in most recent prior election
# seat: seats (0-58.8) seats in most recent prior election (.8 -> coalition fraction???)
# electionyear: 2013|2014|2015|2016|2017
CHES_2017["electionyear"] = CHES_2017["electionyear"].astype(int)

# eumember: not present
CHES_2017["eumember"] = "Yes"

# family: party family
# govt: 0/0.5/1.0
# eu_position: 1-7
# eu_salience:0-10
# eu_dissent:0-10

# eu_asylum:1-7 1NULL
# eu_budgets: 1-7 1NULL
# lrgen,lrecon,galtan: 0-10 
# lrecon_clear, galtan_clear(1NULL): 0-10
# lrecon_salience, galtan_salience: 0-10
# galtan_new, galtan_new_clear(1NULL), galtan_new_salience: 0-10

# policies: all fine 0-10
# 'immig', 'immig_salience', 'immig_dissent',
# 'multicult', 'multicult_salience', 'multicult_dissent', 'minor',
# 'people_vs_elite', 'antielite_salience', 'corrupt_salience', 'members_vs_leadership'


describe_messy_column(CHES_2017["galtan_new_salience"])
# CHES_2017["govt"].value_counts()
# CHES_2017["country"].cat.categories
# CHES_2017.shape,CHES_2017.columns





0  nulls


Series([], Name: galtan_new_salience, dtype: int64)

min:  3.0 max:  9.733334


In [2339]:
CHES_1999_2014[CHES_1999_2014["year"]==1999].shape,\
    CHES_1999_2014[CHES_1999_2014["year"]==2002].shape,\
    CHES_1999_2014[CHES_1999_2014["year"]==2006].shape,\
    CHES_1999_2014[CHES_1999_2014["year"]==2010].shape,\
    CHES_1999_2014[CHES_1999_2014["year"]==2014].shape,

((142, 73), (171, 73), (188, 73), (203, 73), (245, 73))

In [2340]:
# 2014_CHES_dataset_means-2.dta             268
# 2010_CHES_dataset_means.dta               239
# 2014_Candidates_Ukraine_Means.dta          62
# 2007_CHES-candidates_dataset_means.dta     39


In [2341]:
## quickly grab the expert-level datasets
filename = "1999_CHES_dataset_expert-level.dta"
CHES_1999_explvl = pd.read_stata(dataset_folder+"CHES"+os.sep+"expert_level" +os.sep+filename)
CHES_1999_explvl["source"] = filename

filename = "2002_CHES_dataset_expert-level_east.dta"
CHES_2002_explvl_east = pd.read_stata(dataset_folder+"CHES"+os.sep+"expert_level" +os.sep+filename)
CHES_2002_explvl_east["source"] = filename

filename = "2002_CHES_dataset_expert-level_west.dta"
CHES_2002_explvl_west = pd.read_stata(dataset_folder+"CHES"+os.sep+"expert_level" +os.sep+filename)
CHES_2002_explvl_west["source"] = filename

filename = "2006_CHES_dataset_expert-level.dta"
CHES_2006_explvl = pd.read_stata(dataset_folder+"CHES"+os.sep+"expert_level" +os.sep+filename)
CHES_2006_explvl["source"] = filename

filename = "2010_CHES_dataset_expert-level.dta"
CHES_2010_explvl = pd.read_stata(dataset_folder+"CHES"+os.sep+"expert_level" +os.sep+filename)
CHES_2010_explvl["source"] = filename

filename = "2014_CHES_dataset_expert-level.dta"
CHES_2014_explvl = pd.read_stata(dataset_folder+"CHES"+os.sep+"expert_level" +os.sep+filename)
CHES_2014_explvl["source"] = filename

filename = "combined_experts_2017.dta"
CHES_2017_explvl = pd.read_stata(dataset_folder+"CHES"+os.sep+"expert_level" +os.sep+filename)
CHES_2017_explvl["source"] = filename

filename = "2014_Candidates_Ukraine_Experts.dta"
CHES_2014_UKR_explvl = pd.read_stata(dataset_folder+"CHES"+os.sep+"expert_level" +os.sep+filename)
CHES_2014_UKR_explvl["source"] = filename

filename = "2007_CHES-candidates_dataset_expert-level.dta"
CHES_2007_CAND_explvl = pd.read_stata(dataset_folder+"CHES"+os.sep+"expert_level" +os.sep+filename)
CHES_2007_CAND_explvl["source"] = filename

# 1984-1999 and 2019 expert level datasets missing

In [2342]:
for ds in [CHES_1999_explvl,CHES_2002_explvl_east,CHES_2002_explvl_west,
           CHES_2006_explvl,CHES_2010_explvl,CHES_2014_explvl,
           CHES_2017_explvl,CHES_2014_UKR_explvl,CHES_2007_CAND_explvl]:
    continue
#    print(ds["source"].unique()[0],"\n",ds.columns)

# it's a bit messy and there isn't really a shortcut to extracting info

In [2343]:
df = pd.concat([CHES_1984_1999,CHES_1999,CHES_2002,CHES_2006,CHES_2010,CHES_2014,CHES_2007_CAND,CHES_2014_UKR,CHES_2017,CHES_2019],axis=0)
# df.isnull().sum().head(50)

df["uid"] = (df["year"].astype(int).astype(str)+"_"+df["party_id"].astype(int).astype(str))

In [2344]:
df["source"][df["salience2_sd"].notnull()].value_counts()

2006_CHES_dataset_means.dta               187
2007_CHES-candidates_dataset_means.dta     39
Name: source, dtype: int64

In [2345]:
# CHES_2006["salience2_sd"].value_counts().sort_index()



In [2346]:
# Country Harmonisation

ches_country_code_dict = {1: 'be', 2: 'den', 3: 'ger', 4: 'gre', 5: 'spa', 6: 'fra', 7: 'ire', 8: 'it', 10: 'net', 11: 'uk',
 12: 'por', 13: 'aus', 14: 'fin', 16: 'swe', 20: 'bul', 21: 'cze', 22: 'est', 23: 'hun', 24: 'lat', 25: 'lith', 26: 'pol',
 27: 'rom', 28: 'slo', 29: 'sle', 30: 'bih', 31: 'cro', 32: 'mkd', 33: 'ser', 34: 'tur', 35: 'nor', 36: 'swi', 37: 'mal',
 38: 'lux', 40: 'cyp', 41: 'alb', 42: 'kos', 43: 'mne', 44: 'ukr'}

trans_to_party_facts_countries = {"den":"dnk","ger":"deu","gre":"grc","spa":"esp","ire":"irl","it":"ita",
"net":"nld","uk":"gbr","por":"prt","aus":"aut","bul":"bgr","lat":"lva",
"lith":"ltu","pol":"prt","rom":"rou","slo":"svk","sle":"svn","cro":"hrv",
"ser":"srb","swi":"che","mal":"mlt","kos":"xkx",
"be":"bel","be-wall":"bel","be-fl":"bel",
"cz":"cze","czech":"cze",
"hung":"hun","slovak":"svk","sloven":"svn","neth":"nld","nl":"nld",
"ice":"isl","fr":"fra","ge":"deu","gr":"grc","dk":"dnk","sv":"swe",}

def country_harmonize(x):
    if could_be_int(x):
        x = int(x)
    if isinstance(x, int):
        x = ches_country_code_dict[x]
    if x in trans_to_party_facts_countries.keys():
        x = trans_to_party_facts_countries[x]
    return x



partyfacts_countries = pd.read_csv(dataset_folder+"PartyFacts"+os.sep+ "partyfacts-countries.csv")
partyfacts_countries["country"] = partyfacts_countries["country"].apply(lambda x: x.lower())
partyfacts_countries = partyfacts_countries.set_index("country")

df["country"] = df["country"].apply(country_harmonize)
df["country_name"] = df["country"].apply(lambda x: partyfacts_countries.loc[x,"name"])
df["country_continent"] = df["country"].apply(lambda x: partyfacts_countries.loc[x,"continent"])
df["country_region"] = df["country"].apply(lambda x: partyfacts_countries.loc[x,"region"])

def lower_if_not_float(x):
    return x.lower() if not isinstance(x,float) else x

df["country_sub_region"] = np.nan
df.loc[df["country_abb"].apply(lambda x: lower_if_not_float(x))=="be-wall","country_sub_region"] = "Wallonia"
df.loc[df["country_abb"].apply(lambda x: lower_if_not_float(x))=="be-fl","country_sub_region"] = "Flanders"

# drop these! "country_abb","cname"
df = df.drop(["country_abb","cname"],axis=1)

In [2347]:
salience1_dict = {"no importance, never mentioned":1,
 "minor issue":2,
 "important issue":3,
 "one of the most important issues":4,
 "the most important issue for the party":5,
}

df["salience1"] = ( df["salience1"].replace(salience1_dict).astype(float) - 1)/4

salience2_dict = {"little importance":2,
 "some importance":3,
 "great importance":4,}

df["salience2"] = (df["salience2"].replace(salience2_dict).astype(float)-1)/3

df["eu_salience_sd"] = df["salience2_sd"]/3

df["eu_salience"] = df["eu_salience"].astype(float)/10.0

df.loc[df["salience1"].notnull(),"eu_salience"] = df.loc[df["salience1"].notnull(),"salience1"]
df.loc[df["salience2"].notnull(),"eu_salience"] = df.loc[df["salience2"].notnull(),"salience2"]

df = df.drop(["salience1","salience2","salience2_sd"],axis=1,)
df["eu_salience"] = df["eu_salience"].astype(float)

In [2348]:

# dissent1, dissent2, dissent3, dissent3_sd, eu_dissent

# 1-5
dissent1_dict = {"complete unity":1,"minor dissent":2,'significant dissent':3, 'evenly split':4}

df["dissent1"] = ( df["dissent1"].replace(dissent1_dict).astype(float) - 1)/4

# dissent2 1-10
df["dissent2"] = (df["dissent2"]-1)/9

# dissent3 0-10
dissent3_dict = {"party was completely united":0}
df["dissent3"] = df["dissent3"].replace(dissent3_dict).astype(float)/10.0
df["eu_dissent_sd"] = df["dissent3_sd"]/10.0
# eu_dissent 0-10
df["eu_dissent"] = df["eu_dissent"]/10.0

df.loc[df["dissent1"].notnull(),"eu_dissent"] = df.loc[df["dissent1"].notnull(),"dissent1"]
df.loc[df["dissent2"].notnull(),"eu_dissent"] = df.loc[df["dissent2"].notnull(),"dissent2"]
df.loc[df["dissent3"].notnull(),"eu_dissent"] = df.loc[df["dissent3"].notnull(),"dissent3"]

df = df.drop(["dissent1","dissent2","dissent3","dissent3_sd"],axis=1,)
df["eu_dissent"] = df["eu_dissent"].astype(float)

In [2349]:
req_dict = {"neutral":4}

df["eu_econ_req"] = (df["eu_econ_req"].replace(req_dict).astype(float)-1)/6.0
df["eu_pol_req"] = (df["eu_pol_req"].replace(req_dict).astype(float)-1)/6.0
df["eu_good_gov"] = (df["eu_good_gov"].replace(req_dict).astype(float)-1)/6.0

df["eu_econ_require"] = df["eu_econ_require"]/10.0
df["eu_political_require"] = df["eu_political_require"]/10.0
df["eu_googov_require"] = df["eu_googov_require"]/10.0

# print(df["eu_econ_req"].min(),df["eu_econ_req"].max(),df["eu_pol_req"].min(),df["eu_pol_req"].max(),df["eu_good_gov"].min(),df["eu_good_gov"].max())
# print(df["eu_econ_require"].min(),df["eu_econ_require"].max(),df["eu_political_require"].min(),df["eu_political_require"].max(),df["eu_googov_require"].min(),df["eu_googov_require"].max())

df.loc[df["eu_econ_require"].notnull(),"eu_econ_req"] = df.loc[df["eu_econ_require"].notnull(),"eu_econ_require"]
df.loc[df["eu_political_require"].notnull(),"eu_pol_req"] = df.loc[df["eu_political_require"].notnull(),"eu_political_require"]
df.loc[df["eu_googov_require"].notnull(),"eu_good_gov"] = df.loc[df["eu_googov_require"].notnull(),"eu_googov_require"]

df = df.drop(["eu_econ_require","eu_political_require","eu_googov_require"],axis=1,)

In [2350]:
CHES_1999_2014 = pd.read_stata(dataset_folder+"CHES"+os.sep+ "1999-2014_CHES_dataset_means-3.dta")
CHES_1999_2014["source"] = "1999-2014_CHES_dataset_means-3.dta"

# describe_messy_column(CHES_1999_2014["galtan_new_salience"])
# CHES_1999_2014["govt"].value_counts()
# CHES_1999_2014["country"].cat.categories
# CHES_1999_2014.shape,CHES_1999_2014.columns

In [2351]:
# CHES_1999_2014.isnull().sum().head(50)

CHES_1999_2014["uid"] = (CHES_1999_2014["year"].astype(int).astype(str)+"_"+CHES_1999_2014["party_id"].astype(int).astype(str))

In [2352]:
# shared_cols = set.intersection(set(CHES_1999_2014.columns),set(df.columns))
# temp = df[shared_cols].isnull().sum() - CHES_1999_2014[shared_cols].isnull().sum()
# temp[temp>59]

# year, source, party_id
# govt, eumember, party, eastwest, expert, vote

# eu_salience, eu_dissent -> being fixed separately, hopefully!

# cmp_id,seat,electionyear,epvote,


In [2353]:
# CHES_1999_2014[["uid", "govt", "eumember", "party", "eastwest", "expert", "vote",
#                         "cmp_id","seat","electionyear","epvote"]]#.set_index("uid")

In [2354]:


cols = ["uid", "govt", "eumember", "party", "eastwest", "expert", "vote",
                        "cmp_id","seat","electionyear","epvote"]
temp = CHES_1999_2014[cols]
temp.columns = "trend_"+temp.columns

df = df.merge(right=temp, how='left', left_on="uid", right_on="trend_uid",)

In [2355]:
df[["govt","trend_govt"]].drop_duplicates()

govt_dict = {"in government":1.0,"not in government":0.0}
govt_null = df["govt"].isnull()
df.loc[govt_null,"govt"] = df.loc[govt_null,"trend_govt"].replace(govt_dict)


In [2356]:
# eastwest_dict = {"east":0.0,"west":1.0,"western europe":1.0}
# eastwest_null = df["eastwest"].isnull()
# df.loc[eastwest_null,"eastwest"] = df.loc[eastwest_null,"trend_eastwest"]

# df["eastwest"] = df["eastwest"].replace(eastwest_dict)

# df["eastwest"].value_counts()

In [2357]:
# no disagreement
expert_null = df["expert"].isnull()
df.loc[expert_null,"expert"] = df.loc[expert_null,"trend_expert"]

In [2358]:
# some disagreement

# df[["vote","trend_vote"]].drop_duplicates()
# temp = (df["vote"]-df["trend_vote"])
# temp[temp.abs()>5]
# df.loc[[288,442]][["year","country","party_id","vote","trend_vote","party","trend_party","electionyear"]]
# after some basic testing, looks like vote in the trend file is better

vote_non_null = df["trend_vote"].notnull()
df.loc[vote_non_null,"vote"] = df.loc[vote_non_null,"trend_vote"]

df["cmp_id"] = df["trend_cmp_id"]
df["epvote"] = df["trend_epvote"]

In [2359]:
# seat  - no overlap
seat_null = df["seat"].isnull()
df.loc[seat_null,"seat"] = df.loc[seat_null,"trend_seat"]


In [2360]:
df[["electionyear","trend_electionyear"]].drop_duplicates()
# electionary  - no overlap

electionyear_null = df["electionyear"].isnull()
df.loc[electionyear_null,"electionyear"] = df.loc[electionyear_null,"trend_electionyear"]


In [2361]:
eumember_dict = {"eu member state":"Yes","EU member state":"Yes",1:"Yes",0:"No","other":"Other"}
eumember_null = df["eumember"].isnull()

df.loc[eumember_null,"eumember"] = df.loc[eumember_null,"trend_eumember"]
df["eumember"] = df["eumember"].replace(eumember_dict)

df["eumember"].value_counts()

Yes      1834
No        183
Other     128
Name: eumember, dtype: int64

In [2362]:
df[["party","trend_party"]].isnull().sum(axis=1).value_counts()
# leave as is! - I'm alright with two party label columns

1    1443
0     640
2     121
dtype: int64

In [2363]:
# [x for x in df.columns if "trend" in x]
df = df.drop(['trend_uid', 'trend_govt', 'trend_eumember', 'trend_eastwest', 'trend_expert',
         'trend_vote', 'trend_cmp_id', 'trend_seat', 'trend_electionyear', 'trend_epvote'],axis=1)

# drop all but trend_party

In [2364]:
# df.isnull().sum().sort_values().head(50)

# CHES_1999_2014[[x for x in CHES_1999_2014.columns if x not in list(shared_cols)]].isnull().sum()

# 

In [2365]:
# df.merge(right=temp, how='left', left_on="uid", right_on="trend_uid",).isnull().sum().sort_values().tail(50)

In [2366]:
# df.dtypes.value_counts()

# int32 is party_id, of course

# inspected the float64 values - as suspected, precision is spurious but there are a few variables for which would
# (minusculely) change value if reduced - and  a whole bunch not

spuriously_float64_vars = [ 'expert', 'govt', 'enlarge', 'eu_ep_dissent',
       'eu_intmark_dissent', 'eu_employ_dissent', 'eu_agri_dissent',
       'eu_cohesion_dissent', 'eu_environ_dissent', 'eu_asylum_dissent',
       'enlargw_dissent', 'enlarge_dissent', 'electionyear']

df[spuriously_float64_vars]= df[spuriously_float64_vars].astype('float32')



In [2367]:
# df[spuriously_float64_vars].describe() # should all be fine post update

# df.dtypes.value_counts()



# [x for x in df.columns[df.dtypes=='float32'] if (x not in spuriously_float64_vars) and ("sd" not in x)]

float_cols = [
'enviro',
 'enviro_salience',
 'international_security',
 'international_salience',
 'lrecon_salience',
 'galtan_salience',
 'econ_interven',
 'antielite_salience',
 'corrupt_salience',
 'conflict_res',
 'region_status',
 'languages',
 'russia_role',
 'oligarchs',
 'lrecon_clear',
 'galtan_clear',
 'immig_dissent',
 'multicult_dissent',
 'galtan_new',
 'galtan_new_clear',
 'galtan_new_salience',
 'people_vs_elite',
 'members_vs_leadership',
 'eu_blur',
 'lrecon_dissent',
 'lrecon_blur',
 'galtan_dissent',
 'galtan_blur',
 'protectionism',
 'russian_interference',
 'anti_islam_rhetoric',]

df[float_cols] = df[float_cols]/10.0

In [2368]:
# df[second_cols].describe()
# 0-10: first_cols
# 0-10: 

In [2369]:
df["source"][df["eu_budgets"].notnull()].value_counts()

2014_CHES_dataset_means-2.dta    267
CHES2019V3.dta                   246
CHES_means_2017.dta              131
Name: source, dtype: int64

In [2370]:
df["eu_budgets"].max()

6.5384617

In [2371]:
# df["source"][df["enlargw_dissent"].notnull()].value_counts()

In [2372]:
# df["enlarge_dissent"].min(),df["enlarge_dissent"].max()

In [2373]:
# df["enlarge_dissent"].value_counts()

In [2374]:
# df["enlargw_dissent"].value_counts()

In [2375]:
# df.select_dtypes('object').columns

In [2376]:
# # should be strings
# # eumember, country, party, family, 'source', 'partyname1', 'partyname2',
# 'intgroup', 'epgroup',
# 'party_name',
# 'mip_one', 'mip_two', 'mip_three',
# 'uid', 'country_name', 'country_continent',
#        'country_region', 'country_sub_region', 'trend_party'

# # should not be strings
# 'eu_position', 'future',
#        'eu_ep', 'eu_fiscal', 'eu_employ', 'eu_cohesion', 'eu_environ',
#        'eu_asylum', 'eu_foreign','eu_intmark', 'eu_agri','eu_turkey', 
        
 
# 'benefit','membership',

# 'lrgen', 'lrecon', 'galtan',

# 'pubserv',
#        'pubserv_salience', 'dereg', 'dereg_salience', 'redist',
#        'redist_salience', 'civlib', 'civlib_salience', 'lifestyle',
#        'lifestyle_salience', 'religion', 'religion_salience', 'immig',
#        'immig_salience', 'multicult', 'multicult_salience', 'urban',
#        'urban_salience', 'cosmo', 'cosmo_salience', 'decentral',
#        'decentral_salience', 'us', 'us_salience', 'minor', 'minor_salience',



# # 'enlargw',
# df["enlargw"] = df["enlargw"].astype(float)

In [2377]:
# eu_position

# position: 1-7
# {"strongly opposed to ei":1,"neutral, no stance on ei":4,"strongly in favor of ei":7}
# {"strongly in favor":1, "neutral":4, "strongly opposed":7,
#   "strongly opposed to european integration":1,"neutral towards european integration":4,"strongly in favor of european integration":7}
# {"strongly opposed":1,"neutral":4,"strongly in favor":7}
# - std_position



position_dict = {"strongly opposed to ei":1,"neutral, no stance on ei":4,"strongly in favor of ei":7,
                 "strongly in favor":1, "neutral":4, "strongly opposed":7,
                 "strongly opposed to european integration":1,"neutral towards european integration":4,"strongly in favor of european integration":7,
                 "strongly opposed":1,"neutral":4,"strongly in favor":7}

# replace strings and normalise
df["eu_position"] = ( df["eu_position"].replace(position_dict).astype(float) - 1)/6

# amalgamate eu_position standard deviation variables
eu_position_sd_null = df["eu_position_sd"].isnull()
df.loc[eu_position_sd_null,"eu_position_sd"] = df.loc[eu_position_sd_null,"position_sd"]
df = df.drop("position_sd",axis=1)
# OR WE RENAME IT!
# rescale standard deviation
df["eu_position_sd"] = df["eu_position_sd"]/6

# test
df["eu_position"].min(),df["eu_position"].max()

(0.0, 1.0)

In [2378]:
# future: 1-5
# {"unification has gone much too far":1, "just the right amount of unification":3}

future_dict = {"unification has gone much too far":1, "just the right amount of unification":3}

# replace strings and normalise
df["future"] = ( df["future"].replace(future_dict).astype(float) - 1)/4


# test
df["future"].min(),df["future"].max()

(0.0, 0.9583333730697632)

In [2379]:
# Specific EU Policy Questions: 1-7

policy_cols = [ 'eu_ep', 'eu_fiscal', 'eu_employ', 'eu_cohesion', 'eu_environ',
                'eu_asylum', 'eu_foreign','eu_intmark', 'eu_agri','eu_turkey','eu_budgets']
policy_dict = {"strongly opposes expanding ep powers":1,"neutral towards expanding ep powers":4,"strongly favors expanding ep powers":7,
               "strongly opposed expanding ep powers":1,
"strongly opposes tax harmonization":1,"neutral towards tax harmonization":4,"strongly favors tax harmonization":7,
"strongly opposes a common employment policy":1,"neutral towards a common employment policy":4,
"strongly opposes the eu's cohesion policy":1,"neutral towards the eu's cohesion policy":4,"strongly favors the eu's cohesion policy":7,
'neutral':4,'strongly favors':7,'strongly favors ':7,99.0:np.nan,
"strongly opposes a common policy on environment":1,"neutral towards a common policy on environment":4,"strongly favors a common policy on environment":7,
"strongly opposes a common policy on political asylum":1,"neutral towards a common policy on political asylum":4,"strongly favors a common policy on political asylum":7,
"strongly opposes a common foreign and security policy":1,"neutral towards a common foreign and security policy":4,"strongly favors a common foreign and security policy":7,
"strongly opposes":1,"neutral":4,
'stongly opposes expanding EU powers on the internal market':1.0, 'neutral toward expanding EU powers on the internal market':4.0, 'stongly favors explanding EU powers on the internal market':7.0,
"neutral towards internal market":4,            
'strongly opposes agricultural policy':1.0, 'neutral toward agricultural policy':4.0, 'strongly favors agricultural policy':7.0,}

df[policy_cols] = (df[policy_cols].replace(policy_dict)-1)/6.0

df[policy_cols].describe()

,eu_ep,eu_fiscal,eu_employ,eu_cohesion,eu_environ,eu_asylum,eu_foreign,eu_intmark,eu_agri,eu_turkey,eu_budgets
count,1124.000000,283.000000,450.000000,1367.000000,453.000000,831.000000,1374.000000,1096.000000,171.000000,667.000000,644.000000
mean,0.627329,0.527050,0.588750,0.726654,0.688961,0.569053,0.581879,0.645480,0.637300,0.493514,0.441035
std,0.225945,0.281205,0.236534,0.217913,0.219206,0.266014,0.267909,0.244864,0.211213,0.223466,0.252468
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.500000,0.305556,0.439861,0.633333,0.596667,0.388889,0.366667,0.480000,0.500000,0.361389,0.215336
50%,0.666667,0.611111,0.635833,0.785000,0.740000,0.644444,0.666667,0.709722,0.666667,0.533333,0.473291
75%,0.800000,0.750000,0.777778,0.888889,0.833333,0.777778,0.803258,0.847222,0.788333,0.651591,0.666667
max,1.000000,1.000000,0.966667,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.985000,0.923077


In [2380]:
# "benefit" -> "eu_benefit"

# amalgamate benefit and eu_benefit
eu_benefit_null = df["eu_benefit"].isnull()
df.loc[eu_benefit_null,"eu_benefit"] = df.loc[eu_benefit_null,"benefit"]
df = df.drop("benefit",axis=1)
# OR WE RENAME IT!

benefit_dict = {'benefited':1,'neither benefited nor lost':2,'not benefited':3, 0.5: 2}
df["eu_benefit"] = df["eu_benefit"].replace(benefit_dict).astype(float)
df["eu_benefit"] = (df["eu_benefit"]-1)/2

df["eu_benefit"].min(),df["eu_benefit"].max(),

(0.0, 1.0)

In [2381]:
# df["source"][df["benefit"].notnull()].value_counts()
# Ideology: 0-10
# 'lrgen', 'lrecon', 'galtan',

ideology_dict = {"center":5,"extreme right":10,"traditional/authoritarian":10}
ideology_cols = ["lrgen","lrecon","galtan"]

df[ideology_cols] = df[ideology_cols].replace(ideology_dict)/10.0

ideology_sd_cols = ['lrgen_sd','lrecon_sd','galtan_sd']
df[ideology_sd_cols] = df[ideology_sd_cols]/10.0

df[ideology_cols+ideology_sd_cols].describe()

,lrgen,lrecon,galtan,lrgen_sd,lrecon_sd,galtan_sd
count,2072.000000,1659.000000,1658.000000,644.000000,920.000000,920.000000
mean,0.517276,0.487669,0.517333,0.104202,0.126115,0.142429
std,0.231567,0.213291,0.249248,0.059685,0.063574,0.060335
min,0.000000,0.009091,0.013000,0.000000,0.000000,0.000000
25%,0.340000,0.336182,0.308333,0.068584,0.083422,0.100000
50%,0.533333,0.493000,0.512500,0.089400,0.114000,0.133376
75%,0.700000,0.657143,0.721071,0.122736,0.151700,0.179196
max,1.000000,0.980000,1.000000,0.460112,0.900000,0.424264


In [2382]:
# membership: 1-7

df["membership"] = (df["membership"].replace( {'neutral':4.0} ).astype(float) - 1.0)/6.0
df["membership_sd"] = df["membership_sd"]/6.0

df["membership"].min(),df["membership"].max(),

(0.06666666269302368, 0.9716666539510092)

In [2383]:
# Policy Dimensions

policy_dim_cols = ['pubserv',
       'pubserv_salience', 'dereg', 'dereg_salience', 'redist',
       'redist_salience', 'civlib', 'civlib_salience', 'lifestyle',
       'lifestyle_salience', 'religion', 'religion_salience', 'immig',
       'immig_salience', 'multicult', 'multicult_salience', 'urban',
       'urban_salience', 'cosmo', 'cosmo_salience', 'decentral',
       'decentral_salience', 'us', 'us_salience', 'minor', 'minor_salience']

policy_dim_dict = {"extremely important":10,
"strongly supports more rights for ethnic minorities":0,
"strongly favors assimilation":10,
"strongly opposes strong us leadership in world affairs":0,
"strongly supports religious principles in politics":10,
"strongly opposes more rights for ethnic minorities":10,
"strongly favors redistribution":0,
"strongly advocates nationalism":10,
"strongly opposes liberal policies":10,}

df[policy_dim_cols] = df[policy_dim_cols].replace(policy_dim_dict)/10.0

In [2384]:
## normalise float dtype variables

In [2385]:
# describe_messy_column( df[policy_dim_cols].stack() )
# 


In [2386]:
# df[policy_dim_cols].stack()[df[policy_dim_cols].stack()=="strongly opposes more rights for ethnic minorities"]

In [2387]:
# df["eu_cohesion"].value_counts()

# describe_messy_column(df["eu_fiscal"])

# df["source"][df["minor"].notnull()].value_counts()


In [2388]:
# df["source"][df["membership_sd"].notnull()]
# df["source"][df["membership"].notnull()].value_counts()

In [2389]:
# [x for x in df.columns if "_sd" in x]

In [2390]:
# df[df.columns[df.isnull().sum()==0]].dtypes

# df["year"] = df["year"].astype(int)
# df["year"].value_counts()


# FULLY FILLED IN
# country              object
# party_id              int32
# year                  int32
# source               object
# uid                  object
# country_name         object
# country_continent    object
# country_region       object

# ALMOST COMPLETE
# eu_salience              2
# eu_dissent               7
# eu_position            103
# lrgen                  132

# NOT SO VERY INCOMPLETE
# eastwest               322
# vote                   470
# eumember               530
# lrecon                 545
# galtan                 546
# party                  569
# eu_foreign             830
# eu_cohesion            837
# family                 885
# expert                 943
# govt                   984
# electionyear           984

In [2391]:
# df.dtypes

In [2392]:
CHES_1999_2014[ CHES_1999_2014["uid"].apply(lambda x : x not in df["uid"].values) ]

# 710 in CHES_1999 -> 708 in CHES1999_2014

# difference between DS and SP is genuinely confusing!

# 621, 31630, fr, NC (party_id CHES_2010 = 615)
# 836, NaN, it, PdCl (party_id CHES_2010 = 841)
# 1006, 22952, nl, SGP (party_id CHES_2010 = 1019)

# FIXED

,country,eastwest,eumember,year,expert,party_id,cmp_id,party,vote,seat,...,econ_interven,nationalism,antielite_salience,corrupt_salience,mip_one,mip_two,mip_three,chesversion,source,uid


In [2393]:
# rows presents in merged set not present in trend file (surely missing 1984-1999)

df[ df["uid"].apply(lambda x : x not in CHES_1999_2014["uid"].values) ].groupby(["source"])["year"].count()

source
1984-1999_dataset_means.dta               547
2007_CHES-candidates_dataset_means.dta     39
2010_CHES_dataset_means.dta                36
2014_CHES_dataset_means-2.dta              23
2014_Candidates_Ukraine_Means.dta          62
CHES2019V3.dta                            277
CHES_means_2017.dta                       132
Name: year, dtype: int64

In [2394]:
# df.max()#[df.max()>1]

In [2395]:
df.dtypes.value_counts()

float64    82
float32    28
object     19
int32       1
dtype: int64

In [2396]:
numeric_cols = df.columns[ df.dtypes!='object' ]

In [2397]:
df[numeric_cols].min().sort_values().tail(20)

russia_role                  0.123529
pubserv_salience             0.125000
us_salience                  0.125000
minor_salience               0.133000
members_vs_leadership        0.150000
civlib_salience              0.175000
eu_good_gov                  0.184211
cosmo_salience               0.200000
lifestyle_salience           0.200000
lrecon_clear                 0.207692
lrecon_salience              0.220000
galtan_clear                 0.223077
galtan_new_clear             0.290000
galtan_new_salience          0.300000
oligarchs                    0.406250
expert                       3.000000
party_id                   101.000000
year                      1984.000000
electionyear              1996.000000
cmp_id                   11110.000000
dtype: float64

In [2398]:
# df.select_dtypes('object').columns

In [2399]:
# country, country_name, country_continent, country_region,
# source, year, uid, party_id, partyname1, partyname2, trend_party, 
# eumember, intgroup, epgroup, family, 

# enlargw
# df["enlargw"].max()

In [2400]:
# shared_cols = set.intersection(set(CHES_1999_2014.columns),set(df.columns))
# temp = df[shared_cols].isnull().sum() - CHES_1999_2014[shared_cols].isnull().sum()
# temp[temp>59]

In [2401]:
shared_cols = set.intersection(set(CHES_1999_2014.columns),set(df.columns))

In [2402]:
temp = df[shared_cols].notnull().sum() - CHES_1999_2014[shared_cols].notnull().sum()

In [2403]:
temp[temp<=0]
# no more information in trend file that we haven't extracted (at least, on easily matched columns!)

us_salience       0
us                0
eu_agri           0
cosmo_salience    0
dtype: int64

In [2404]:
numeric_cols = df.columns[ df.dtypes!='object' ]

corrs = df[numeric_cols].corr()
corrs = corrs.where(np.triu(np.ones(corrs.shape),k=1).astype(np.bool))
corr_df = corrs.stack().reset_index()
corr_df.columns = ['Row','Column','Value']
# df.drop(df.index[df["Value"]==1.0],inplace=True)

corr_df["N"] = corr_df.apply(lambda x: (df[numeric_cols][x["Row"]].notnull()&df[numeric_cols][x["Column"]].notnull()).sum(),axis=1)

corr_df.sort_values(by="Value")

,Row,Column,Value,N
3497,region_status,languages,-0.989737,12
3495,conflict_res,russia_role,-0.987668,12
3494,conflict_res,languages,-0.987145,12
3257,eu_pol_req,members_vs_leadership,-0.962572,5
3498,region_status,russia_role,-0.962091,12
...,...,...,...,...
348,vote,seat,0.970336,1141
265,year,electionyear,0.981751,1220
3493,conflict_res,region_status,0.982203,12
3500,languages,russia_role,0.982498,12


In [2405]:
corr_df[corr_df["N"]>100].sort_values(by="Value").head(5)

,Row,Column,Value,N
424,eu_position,eu_benefit,-0.940574,678
841,eu_foreign,eu_benefit,-0.891185,674
3109,eu_benefit,eu_budgets,-0.884501,267
1338,eu_intmark,eu_benefit,-0.850612,678
753,eu_asylum,cosmo,-0.824235,246


In [2406]:
corr_df[corr_df["N"]>100].sort_values(by="Value").tail(5)

# I looked deeper on both, but it all looks reasonable
# still don't know what govt2/govt3 are - but they're very nearly the same variable

,Row,Column,Value,N
363,eu_position,future,0.966190,283
1142,galtan,galtan_new,0.969702,132
348,vote,seat,0.970336,1141
265,year,electionyear,0.981751,1220
1176,govt,govt1,0.986555,188


In [2407]:
# df.isnull().sum().sort_values()

In [2408]:
# df["source"][df["eumember"].isnull()].value_counts()

# eu_position
# candidates in 2007, 2014 not asked 'position' question on integration
# but instead 'membership' question on joining - and they're really not at all identical

# ... but the wording for 'eu_salience' doesn't change - should they be treated as same variable
# ASK

In [2409]:
# df[df["lrgen"].isnull()][df.columns[0:20]]

In [2410]:
CHES_2007_CAND.shape,CHES_2014_UKR.shape,CHES_2017.shape,CHES_2019.shape,CHES_2010.shape

# eumember

((39, 30), (62, 40), (132, 38), (277, 57), (239, 47))

In [2411]:
# drop eastwest from previous processing - no need!


# EASTWEST = 0: party from Central/Eastern Europe, 1: party from EU-15
        

eu15 = ["lux","ita","nld","prt","bel",
"dnk","fin","fra","deu","grc",
"irl","esp","swe","gbr","aut",]

# 2019 dataset includes Malta and Cyprus as "west" in "eastwest" ... despite defining it as "EU-15"
# (which would be a clearer variable name and which they're not!)

# ASK!

df["eastwest"] = df["country"].apply(lambda x: x in eu15).astype(float)
df = df.rename(columns = {"eastwest":"eu15"})

# 2010_CHES_dataset_means.dta           36
# 2014_CHES_dataset_means-2.dta         23

# 2014
# che    11
# nor     8
# tur     4

# 2010
# che    12
# hrv    11
# nor     7
# tur     6

eumember_2010_2014_dict = {"che":"No","hrv":"Other","nor":"No","tur":"Other"}

df.loc[df["eumember"].isnull(),"eumember"] = df.loc[df["eumember"].isnull(),"country"].replace( eumember_2010_2014_dict )




In [2412]:
# CHES_2007_CAND["eumember"].value_counts()



In [2413]:
# df[["party_id","cmp_id"]].drop_duplicates().dropna()["party_id"].value_counts().head(20)


# # cmp_id ==81713 x2
# # party_id
# 306     3
# 2505    2
# 2507    2
# 110     2
# 112     2
# 609     2
# 2704    2
# 403     2
# 2904    2
# 2903    2
# 2902    2
# 815     2
# 814     2
# 106     2

In [2414]:
df[df["cmp_id"]==81713][['party_id', 'party', 'partyname1', 'partyname2', 'party_name', 'trend_party','cmp_id']]

# CHES_2014 codebook:
# 3109 HSP Hrvatska Stranka Prava Croatian Party of Rights
# https://en.wikipedia.org/wiki/Croatian_Party_of_Rights
# ...
# 3113 HSP-AS Hrvatska stranka prava dr. Ante Starcevic Croatian Party of Rights dr. Ante Starcevic
# https://en.wikipedia.org/wiki/Croatian_Party_of_Rights_Dr._Ante_Star%C4%8Devi%C4%87

# got it - HSP-AS is a splinter party from HSP - performs slightly better than HSP

# CMP codebook:
# 81713 1992-2003 HSP Hrvatska stranka prava (Croatian Party of Rights) 4
# 81061 2015 Alliance: Domoljubna koalicija (Patriotic Coalition);
# members: HDZ (81711), HSS (81810), HSLS (81410), HSP
# AS Hrvatska stranka prava dr. Ante Starčević (Croatian
# Party of Rights dr. Ante Starčević), Hrast Hrvatski rast
# (Croatian Growth), BUZ Blok umirovljenici zajedno (Bloc
# Pensioners Together), HDS Hrvatska kršćanska
# demokratska stranka (Croatian Christian Democratic
# Party)

,party_id,party,partyname1,partyname2,party_name,trend_party,cmp_id
1653,3109,NaN,NaN,NaN,HSP,HSP,81713.0
1655,3113,NaN,NaN,NaN,HSP-AS,HSP-AS,81713.0


In [2415]:
# temp = df[["party_id","cmp_id"]].drop_duplicates().dropna()["party_id"].value_counts()
# # temp = df[df["party_id"].apply(lambda x: x in temp[temp>1].index)][['party_id', 'party', 'partyname1', 'partyname2', 'party_name', 'trend_party','cmp_id',"source"]].drop_duplicates().tail(50)

# # temp[temp["cmp_id"].notnull()].sort_values(by='party_id')

# # done - there are several overlaps (multiple cmp_ids for same party_id) - but they all relate to legitimate splits/mergers

# temp[temp>1]

# df[df["party_id"].apply(lambda x: x in temp[temp>1].index)][["party_id","cmp_id"]].drop_duplicates().dropna().sort_values(by='party_id')


In [2416]:
df[["party_id","year"]]=df[["party_id","year"]].astype('uint16')

In [2417]:
variable_dict = {}

# EU integration/joining
cols =['eu_position', 'eu_salience', 'eu_dissent', 'eu_benefit', 'eu_blur',
       'eu_position_sd', 'eu_salience_sd', 'eu_dissent_sd', 'membership', 'membership_sd', 'future',]
variable_dict.update({k:"eu_integration_admission" for k in cols})

# Specific EU Policy Questions
cols =['eu_ep', 'eu_fiscal', 'eu_employ',
       'eu_cohesion', 'eu_environ', 'eu_asylum', 'eu_foreign',
       'eu_intmark', 'eu_agri', 'eu_turkey', 'eu_budgets', 'enlargw', 'enlarge',]
variable_dict.update({k:"eu_policy_questions" for k in cols})    
        
# EU membership requirements       
cols =['eu_econ_req', 'eu_pol_req', 'eu_good_gov',]
variable_dict.update({k:"eu_membership_requirements" for k in cols})
    
# Ideological Questions    
cols =['lrgen', 'lrecon', 'galtan', 
       'lrecon_salience', 'galtan_salience',
       'lrecon_clear', 'galtan_clear',
       'lrecon_dissent', 'lrecon_blur', 'galtan_dissent',
       'galtan_blur', 'galtan_new', 'galtan_new_clear', 'galtan_new_salience',
       'lrgen_sd', 'lrecon_sd','galtan_sd',]
variable_dict.update({k:"ideological_dimensions" for k in cols})
    
# Policy Dimensions    
cols =['pubserv', 'pubserv_salience', 'dereg', 'dereg_salience', 'redist',
       'redist_salience', 'civlib', 'civlib_salience', 'lifestyle',
       'lifestyle_salience', 'religion', 'religion_salience', 'immig',
       'immig_salience', 'multicult', 'multicult_salience', 'urban',
       'urban_salience', 'cosmo', 'cosmo_salience', 'decentral',
       'decentral_salience', 'us', 'us_salience', 'minor', 'minor_salience',
       'enviro', 'enviro_salience', 'international_security', 'international_salience', 
       'econ_interven', 'immig_dissent', 'multicult_dissent',
       'protectionism', 'russian_interference', 'anti_islam_rhetoric',]
variable_dict.update({k:"policy_dimensions" for k in cols})
        
# Party characteristics
cols =['antielite_salience', 'corrupt_salience',  'people_vs_elite', 'members_vs_leadership',]
variable_dict.update({k:"party_characteristics" for k in cols})    

# Ukraine-specific questions    
cols =['conflict_res', 'region_status', 'languages', 'russia_role', 'oligarchs',]
variable_dict.update({k:"ukraine_specific_questions" for k in cols})

# Leadership divisions (fraction of experts say this issue caused divisions)    
cols =['eu_ep_dissent', 'eu_intmark_dissent', 'eu_employ_dissent', 'eu_agri_dissent',
       'eu_cohesion_dissent', 'eu_environ_dissent', 'eu_asylum_dissent',
       'enlargw_dissent', 'enlarge_dissent',]
variable_dict.update({k:"leadership_divisions" for k in cols})

In [2418]:
# Observation variables
cols =['source', 'uid', 'party_id', 'cmp_id', 'year', 'expert',]
variable_dict.update({k:"observation_variables" for k in cols})

# Party Name variables
cols =['party', 'partyname1', 'partyname2', 'party_name', 'trend_party']
variable_dict.update({k:"party_names" for k in cols})

# Party Electoral variables
cols =['govt', 'govt1', 'govt2', 'govt3', 'epvote', 'vote', 'vote1', 'seat', 'electionyear',]
variable_dict.update({k:"party_electoral" for k in cols})

# Country variables (aligned with/from partyfacts)
cols =['country', 'country_name', 'country_continent', 'country_region', 'country_sub_region', 'eu15',]
variable_dict.update({k:"country_variables" for k in cols})

# dummiable
cols= ['eumember', 'family', 'intgroup', 'epgroup', 'mip_one', 'mip_two', 'mip_three', ]
variable_dict.update({k:"dummiable_variables" for k in cols})


In [2419]:
family_replace_dict = {"1. radical TAN":"rad right",
 "2. conservative":"cons",
 "3. liberal":"liberal",
 "4. Christian-democratic":"christdem",
 "5. socialist":"socialist",
 "6. radical left":"rad left",
 "7. green":"green",
 "8. regionalist":"regionalist",
 "9. no family":"no family",
 "10. confessional":"confessional",
 "11. agrarian/center":"agrarian",
 "agrarian/centre":"agrarian",
 "rad right ":"rad right",
 "protestant":"confessional", # not 100% certain about this! theoretically, you can have confessional parties that aren't prot
 "regional/ethnic":"regionalist",
 "region/ethnic":"regionalist",
 "radical right":"rad right",
 "radical left":"rad left",
}

df["family"] = df["family"].replace(family_replace_dict)

In [2420]:
### all to get nicely consistent dummies most important issue variables
dummy_df = pd.DataFrame()

mip_replace_dict = {"public services vs taxes":"services vs taxes",
                    "anti-elite rhetoric":"anti-elite",
                    "ethnic minorities":"ethnic", # change these back to long-form later
                    "EU integration":"EU",# change these back to long-form later
                   }

# all the values up until the "tie: " combinations
base_list = list(df[["mip_one","mip_two","mip_three"]].replace(mip_replace_dict).stack().value_counts().index[0:18])

base_list = pd.Series(base_list).replace(mip_replace_dict).values

df[["mip_one","mip_two","mip_three"]] = df[["mip_one","mip_two","mip_three"]].replace(mip_replace_dict)

def mip_transform(x):
    if pd.isnull(x):
        return np.nan
    elif "tie:" in x:
        return x.replace("tie: ","").split(" and ")
    else:
        return [x]

long_form_dict ={"EU":"EU integration","ethnic":"ethnic minorities"}
    
for cat in base_list:
    for mip_pos in ["one","two","three"]:
        if cat in long_form_dict.keys():
            long_cat = long_form_dict[cat]
        else:
            long_cat = cat
        dummy_df["mip_"+mip_pos+"__"+long_cat] = df["mip_"+mip_pos]\
                                    .apply(lambda x: mip_transform(x))\
                                    .apply(lambda x: np.nan if x is np.nan else cat in x)\
                                    .astype('float32')

df[dummy_df.columns] = dummy_df        
df = df.drop(["mip_one","mip_two","mip_three"],axis=1)
#####

dummy_df = pd.get_dummies(df["family"],prefix="family", prefix_sep='__')
dummy_df.loc[df["family"].isnull(),:] = np.nan
df[dummy_df.columns] = dummy_df
#####

dummy_df = pd.get_dummies(df["eumember"],prefix="eumember", prefix_sep='__')
dummy_df.loc[df["eumember"].isnull(),:] = np.nan
df[dummy_df.columns] = dummy_df
#####

epgroup_dict = {'pes':"Party of European Socialists (PES)",'epp-ed':"European People’s Party – European Democrats (EPP-ED)",
'alde':"Alliance for Liberals and Democrats for Europe (ALDE)",'greens-efa':"European Greens – European Free Alliance (Greens-EFA)",
'uen':"Union for Europe of the Nations (UEN)",'gue-ngl':"European United Left – Nordic Green Left (GUE-NGL)",
'id':"Independence/Democracy (ID)",'non-incrits':"Non-Incrits (NI)",8.0:"no group"}

dummy_df = pd.get_dummies(df["epgroup"].replace(epgroup_dict),
                          prefix="epgroup", prefix_sep='__')
dummy_df.loc[df["epgroup"].isnull(),:] = np.nan
df[dummy_df.columns] = dummy_df


def set_dummy(ser,cat):
    x = ser.copy()
    x.loc[ser==cat] = 1.0
    x.loc[ser!=cat] = 0.0
    x.loc[ser.isnull()] = np.nan
    if cat in ["centrist democratic international","international democrat union"]:
        x.loc[ser=="cdi (2) and idu (5)"] = 1.0
        
    return x

base_list = ['alliance of democrats', 'centrist democratic international', 'global greens',
             'international democrat union', 'liberal international',
             'no affiliation', 'socialist international'] 

for cat in base_list:
    df[ "intgroup__" + cat ] = set_dummy( df["intgroup"] , cat )

In [2423]:
# dummied variables
cols = [x for x in df.columns if "__" in x]
variable_dict.update({k:"dummied_variables" for k in cols})

In [2452]:
dup_uids = df["uid"].value_counts()[ df["uid"].value_counts()>1 ].index
dup_uids2 = df[df["uid"].apply(lambda x: x in dup_uids)][df.drop(["source","party"],axis=1).columns].drop_duplicates().sort_values(by="uid")["uid"].value_counts()

dup_uids2[dup_uids2>1]

Series([], Name: uid, dtype: int64)

In [2460]:
df = df.drop( df[df["uid"].apply(lambda x: x in dup_uids) & (df["source"]=='1984-1999_dataset_means.dta')].index )

In [2461]:
df["uid"].value_counts()

## finally! uid now *actually* a unique identifier

2019_1606    1
2006_105     1
2006_802     1
2019_3604    1
2017_2619    1
            ..
2006_2501    1
1988_1210    1
2002_2305    1
1988_1107    1
2014_1016    1
Name: uid, Length: 2065, dtype: int64

In [2463]:
df.columns

Index(['eumember', 'country', 'expert', 'party_id', 'year', 'party', 'vote',
       'family', 'eu_position', 'future',
       ...
       'epgroup__Party of European Socialists (PES)',
       'epgroup__Union for Europe of the Nations (UEN)', 'epgroup__no group',
       'intgroup__alliance of democrats',
       'intgroup__centrist democratic international',
       'intgroup__global greens', 'intgroup__international democrat union',
       'intgroup__liberal international', 'intgroup__no affiliation',
       'intgroup__socialist international'],
      dtype='object', length=212)

In [2464]:
df.shape

(2065, 212)

In [2469]:
df.to_pickle(dataset_folder+"CHES"+os.sep+"CHES_comb.zip",compression='zip')
df.to_csv(dataset_folder+"CHES"+os.sep+"CHES_comb.csv")

# dta save throws up error due to 'latin-1' codec can't encode character '\u2013' in position 4: ordinal not in range(256)
# in 'party'